In [0]:
import re
from pathlib import Path
from pyspark.sql.types import *

def sql_to_databricks_delta(sql_file_path, table_name=None, schema_name='nip_refined'):
    """
    Converts a SQL Server CREATE TABLE statement file to a Databricks CREATE OR REPLACE TABLE statement for Delta.

    Parameters:
    - sql_file_path: Path to the .sql file containing the CREATE TABLE statement.
    - table_name: Optional override for the table name.

    Returns:
    - Databricks SQL CREATE OR REPLACE TABLE statement as a string.
    """
    # Read SQL file
    sql_text = Path(sql_file_path).read_text()
    precision = None
    scale = None

    # Extract table name
    table_match = re.search(r'CREATE\s+TABLE\s+\[?(\w+)\]?', sql_text, re.IGNORECASE)
    tbl = table_name or (table_match.group(1) if table_match else "unknown_table")

    # Extract column definitions
    columns_block = re.search(r'\((.*?)\)\s*ON\s*\[PRIMARY\]', sql_text, re.DOTALL)
    if not columns_block:
        raise ValueError("Could not find column definitions in SQL file.")
    columns_text = columns_block.group(1)

    # Map SQL Server types to Databricks types
    type_map = {
        "int": "INT",
        "bigint": "BIGINT",
        "smallint": "SMALLINT",
        "tinyint": "TINYINT",
        "bit": "BOOLEAN",
        "float": "DOUBLE",
        "real": "FLOAT",
        "decimal": "DECIMAL",
        "numeric": "DECIMAL",
        "money": "DECIMAL(19,4)",
        "smallmoney": "DECIMAL(10,4)",
        "char": "STRING",
        "varchar": "STRING",
        "nchar": "STRING",
        "nvarchar": "STRING",
        "text": "STRING",
        "ntext": "STRING",
        "date": "DATE",
        "datetime": "TIMESTAMP",
        "datetime2": "TIMESTAMP",
        "smalldatetime": "TIMESTAMP",
        "time": "STRING",
        "uniqueidentifier": "STRING",
        "binary": "BINARY",
        "varbinary": "BINARY",
        "image": "BINARY",
        "xml": "STRING",
        "timestamp": "BINARY"
    }

    # Parse columns
    columns = []
    for line in columns_text.splitlines():
        col_match = re.match(r'\s*\[?(\w+)\]?\s+\[?([\w\(\), ]+)\]?(?:\s+\w+)?(?:\s+NULL|\s+NOT\s+NULL)?\s*,?', line)

        if col_match:
            col_name = col_match.group(1)
            sql_type = col_match.group(2).strip().lower()
            base = re.match(r'(\w+)', sql_type)
            if not base:
                columns.append(f"{col_name} STRING")
            
            base_type = base.group(1)
            if base_type in ["decimal", "numeric"]:
                precision_scale = re.search(r'\((\d+),\s*(\d+)\)', line)
                if precision_scale:
                    columns.append(f"{col_name} DECIMAL({precision_scale.group(1)},{precision_scale.group(2)})")
                else:
                    columns.append(f"{col_name} DECIMAL(38,18)")
                
            columns.append(f"{col_name} {base_type}")

    columns_sql = ",\n  ".join(columns)

    # Compose Databricks SQL statement
    dbx_sql = f"""CREATE OR REPLACE TABLE {schema_name}.{tbl} (
  {columns_sql}
)
"""
    return dbx_sql


In [0]:
sql_to_databricks_delta('/Workspace/Users/jn@nipgroup.com/nip-databricks/warehouse/sp_premium_triangles.sql', 'sp_premium_triangles')

In [0]:
type_map = {
        "int": "INT",
        "bigint": "BIGINT",
        "smallint": "SMALLINT",
        "tinyint": "TINYINT",
        "bit": "BOOLEAN",
        "float": "DOUBLE",
        "real": "FLOAT",
        "decimal": "DECIMAL",
        "numeric": "DECIMAL",
        "money": "DECIMAL(19,4)",
        "smallmoney": "DECIMAL(10,4)",
        "char": "STRING",
        "varchar": "STRING",
        "nchar": "STRING",
        "nvarchar": "STRING",
        "text": "STRING",
        "ntext": "STRING",
        "date": "DATE",
        "datetime": "TIMESTAMP",
        "datetime2": "TIMESTAMP",
        "smalldatetime": "TIMESTAMP",
        "time": "STRING",
        "uniqueidentifier": "STRING",
        "binary": "BINARY",
        "varbinary": "BINARY",
        "image": "BINARY",
        "xml": "STRING",
        "timestamp": "BINARY"
    }

In [0]:
%sql

USE CATALOG hive_metastore;
USE SCHEMA nip_refined;

CREATE OR REPLACE TABLE claims(
	as_of DATE,
	book STRING,
	account_name STRING,
	specialty_program STRING,
	carrier STRING,
	LOB STRING,
	accident_am_or_pm STRING,
	accident_address_1 STRING,
	accident_address_2 STRING,
	accident_address_3 STRING,
	accident_city STRING,
	accident_county STRING,
	accident_department STRING,
	accident_description STRING,
	accident_map STRING,
	accident_postal_code STRING,
	accident_shift STRING,
	accident_state STRING,
	accident_time STRING,
	account_id STRING,
	activity_before_loss STRING,
	address_claimant_address_1 STRING,
	address_claimant_address_2 STRING,
	address_claimant_address_3 STRING,
	address_claimant_city STRING,
	address_claimant_postal_code STRING,
	address_claimant_state STRING,
	alternate_claim_number STRING,
	area_of_accident STRING,
	attorney_representation_date DATE,
	basel_business_line STRING,
	basel_loss_event_type STRING,
	benefit_state STRING,
	bodily_injury STRING,
	body_ncci STRING,
	captive_os_expense FLOAT,
	captive_os_indemnity_bi_pd_coll FLOAT,
	captive_os_legal FLOAT,
	captive_os_medical_comp FLOAT,
	captive_os_other FLOAT,
	captive_os_total FLOAT,
	captive_paid_expense FLOAT,
	captive_paid_indem_bi_pd_coll FLOAT,
	captive_paid_legal FLOAT,
	captive_paid_medical_comp FLOAT,
	captive_paid_other FLOAT,
	captive_paid_total FLOAT,
	case_type STRING,
	cause STRING,
	cause_ncci STRING,
	claim_deductible FLOAT,
	claim_number STRING,
	claim_or_incident STRING,
	claim_stage STRING,
	claim_status STRING,
	claimant_death_date DATE,
	claimant_first_name STRING,
	claimant_last_name STRING,
	claimant_m_i STRING,
	claimant_name STRING,
	claimant_title STRING,
	claims_made_date DATE,
	claim_created_date DATE,
	claim_currency_iso_code STRING,
	claim_last_updated_date_char STRING,
	clash_claim STRING,
	class_code STRING,
	closed_date DATE,
	construction_defect STRING,
	controverted STRING,
	coverage STRING,
	coverage_code STRING,
	created_by STRING,
	created_by_data_process STRING,
	created_date DATE,
	created_date_rc_claim DATE,
	TPA STRING,
	date_suit_filed DATE,
	date_placed_in_litigation DATE,
	dates_source_claims_made_date DATE,
	dates_source_close_date DATE,
	dates_source_entry_date DATE,
	dates_source_loss_date DATE,
	dates_source_reopen_date DATE,
	dates_source_report_date DATE,
	days_paid FLOAT,
	dde_copy_data_from_ee_record STRING,
	ded_sir_os_expense FLOAT,
	ded_sir_os_indemnity_bi_pd_coll FLOAT,
	ded_sir_os_legal FLOAT,
	ded_sir_os_medical_comp FLOAT,
	ded_sir_os_other FLOAT,
	ded_sir_os_total FLOAT,
	ded_sir_paid_expense FLOAT,
	ded_sir_paid_indem_bi_pd_coll FLOAT,
	ded_sir_paid_legal FLOAT,
	ded_sir_paid_medical_comp FLOAT,
	ded_sir_paid_other FLOAT,
	ded_sir_paid_total FLOAT,
	deductible_description STRING,
	deductible_from_policy STRING,
	descr_amtrust_spoiled_claim_num STRING,
	descriptors_source_body_part STRING,
	descriptors_source_cause STRING,
	descriptors_source_coverage STRING,
	descriptors_source_location STRING,
	descriptors_source_nature STRING,
	descriptors_source_pol_eff_date DATE,
	descriptors_source_policy_number STRING,
	descriptors_source_policy_period STRING,
	descriptors_source_state STRING,
	descriptors_source_status STRING,
	descriptors_travelers_sai_number STRING,
	descr_travelers_st_paul_pol_num STRING,
	downtime_units STRING,
	driver_age FLOAT,
	driver_birth_date DATE,
	driver_first_name STRING,
	driver_last_name STRING,
	driver_m_i STRING,
	driver_relationship STRING,
	driver_sex STRING,
	driver_title STRING,
	drivers_license_number STRING,
	driver_vehicle_additional_info STRING,
	drivers_first_name STRING,
	employer_notification_date DATE,
	first_aid_care STRING,
	garage_state STRING,
	gross_incurred_expense FLOAT,
	gross_incurred_indem_bi_pd_coll FLOAT,
	gross_incurred_legal FLOAT,
	gross_incurred_medical_comp FLOAT,
	gross_incurred_other FLOAT,
	gross_incurred_total FLOAT,
	healthcare_address_1 STRING,
	healthcare_address_2 STRING,
	healthcare_city STRING,
	healthcare_phone STRING,
	healthcare_postal_code STRING,
	healthcare_provider_type STRING,
	healthcare_state STRING,
	hire_car_required STRING,
	hospitalized_overnight STRING,
	id_for_data_loads STRING,
	in_litigation STRING,
	information_claimant_age FLOAT,
	info_claimant_date_of_birth DATE,
	information_claimant_gender STRING,
	information_claimant_job_title STRING,
	info_claimant_marital_status STRING,
	info_claimant_phone_number STRING,
	information_email STRING,
	information_employee_id STRING,
	information_employee_status STRING,
	information_employment_end_date DATE,
	info_employment_start_date DATE,
	information_hire_date DATE,
	information_job_termination_date DATE,
	information_ncci_class_code STRING,
	info_occupation_when_injured STRING,
	information_supervisor_email STRING,
	information_supervisor_name STRING,
	information_tax_id STRING,
	information_tax_id_admin STRING,
	information_wage_amount FLOAT,
	information_wage_type STRING,
	initial_reserve FLOAT,
	initial_treatment_date DATE,
	injury_or_illness_description STRING,
	insured_name STRING,
	insured_driver_citation STRING,
	insured_driver_information STRING,
	insured_driver_injured STRING,
	insured_driver_injury_desc STRING,
	insured_vehicle_damage_desc STRING,
	insured_vehicle_damaged STRING,
	insurer_os_expense FLOAT,
	insurer_os_indemnity_bi_pd_coll FLOAT,
	insurer_os_legal FLOAT,
	insurer_os_medical_comp FLOAT,
	insurer_os_other FLOAT,
	insurer_os_total FLOAT,
	insurer_paid_expense FLOAT,
	insurer_paid_indem_bi_pd_coll FLOAT,
	insurer_paid_legal FLOAT,
	insurer_paid_medical_comp FLOAT,
	insurer_paid_other FLOAT,
	insurer_paid_total FLOAT,
	jif_incurred FLOAT,
	jif_paid FLOAT,
	last_financial_update DATE,
	last_load_date DATE,
	last_updated_by STRING,
	last_updated_date DATE,
	last_worked_date DATE,
	loss_date DATE,
	loss_description_code STRING,
	lost_days FLOAT,
	lot_number STRING,
	managed_care_fees STRING,
	manufactured_date DATE,
	medical_treatment_sought STRING,
	model_number STRING,
	nature STRING,
	nature_ncci STRING,
	net_incurred_expense FLOAT,
	net_incurred_indem_bi_pd_coll FLOAT,
	net_incurred_legal FLOAT,
	net_incurred_medical_comp FLOAT,
	net_incurred_other FLOAT,
	net_incurred_total FLOAT,
	object_substance_causing_harm STRING,
	occ_severity_code STRING,
	occurrence_lead_claim STRING,
	event_number STRING,
	office_code STRING,
	organization STRING,
	osha_completed_by_rep STRING,
	osha_injury_type STRING,
	osha_privacy STRING,
	osha_recordable STRING,
	osha_rep_phone STRING,
	osha_rep_title STRING,
	part STRING,
	physician_healthcare_name STRING,
	police_professional_liability STRING,
	pol_eff_dt DATE,
	pol_exp_dt DATE,
	policy_number STRING,
	policy_record_id FLOAT,
	policy_year FLOAT,
	primary_insurer_limit STRING,
	product_name STRING,
	program_name STRING,
	property_damage STRING,
	purpose_of_journey STRING,
	ql_case_type STRING,
	recovery_expected_expense FLOAT,
	recov_expected_indem_bi_pd_coll FLOAT,
	recovery_expected_legal FLOAT,
	recovery_expected_medical_comp FLOAT,
	recovery_expected_other FLOAT,
	recovery_expected_total FLOAT,
	recovery_paid FLOAT,
	recovery_received_expense FLOAT,
	recov_received_indem_bi_pd_coll FLOAT,
	recovery_received_legal FLOAT,
	recovery_received_medical_comp FLOAT,
	recovery_received_other FLOAT,
	recovery_received_total FLOAT,
	reimbursement STRING,
	reimbursement_amount FLOAT,
	reinsurance_recovery FLOAT,
	release_to_work_date DATE,
	reopen_date DATE,
	reopened STRING,
	report_date DATE,
	report_lag FLOAT,
	restricted_days FLOAT,
	return_to_work_date DATE,
	road_conditions STRING,
	road_type STRING,
	salvage_indicator STRING,
	salvage_recovery FLOAT,
	serial_number STRING,
	subro_indicator STRING,
	subro_recovery FLOAT,
	suit_date DATE,
	time_employee_began_work STRING,
	total_incurred_net FLOAT,
	total_loss STRING,
	total_outstanding FLOAT,
	total_paid FLOAT,
	total_paid_net FLOAT,
	total_recovery FLOAT,
	town_incurred FLOAT,
	town_paid FLOAT,
	tpa_adjuster STRING,
	treated_in_er STRING,
	user_claim_type_override STRING,
	vehicle_description STRING,
	vehicle_downtime STRING,
	vehicle_make STRING,
	vehicle_model STRING,
	vehicle_number STRING,
	vehicle_tag STRING,
	vehicle_year FLOAT,
	vin STRING,
	visibility STRING,
	weather_conditions STRING,
	work_began_am_or_pm STRING,
	year FLOAT,
	claim_last_updated_date DATE,
	cov_desc STRING,
	cov_desc_detail STRING,
	lob_orig STRING
);

CREATE OR REPLACE TABLE claims_dh(
	as_of DATE,
	public_entity STRING,
	member_name STRING,
	claim_number STRING,
	case_type STRING,
	descriptors_source_policy_number STRING,
	descriptors_source_policy_period STRING,
	insured_name STRING,
	policy_year SMALLINT,
	pol_eff_dt DATE,
	pol_exp_dt DATE,
	policy_number STRING
);

CREATE OR REPLACE TABLE claims_qualynx(
	as_of DATE,
	public_entity STRING,
	member_name STRING,
	policy_number STRING,
	descriptors_source_policy_number STRING,
	insured_name STRING
);

CREATE OR REPLACE TABLE exposures(
	as_of DATE,
	book STRING,
	account_name STRING,
	specialty_program STRING,
	carrier STRING,
	LOB STRING,
	aop_deductible STRING,
	address_1 STRING,
	address_2 STRING,
	building_limit FLOAT,
	building_number STRING,
	city STRING,
	class_code STRING,
	construction STRING,
	construction_class STRING,
	contents_limit FLOAT,
	coverage STRING,
	coverage_reference STRING,
	created_by STRING,
	created_by_data_process STRING,
	exposure_created_date DATE,
	created_date_rc_exp DATE,
	deductible FLOAT,
	deductible_type STRING,
	eq_deductible FLOAT,
	eq_limit FLOAT,
	endorsement_number FLOAT,
	expos_dt DATE,
	expos_eff_dt DATE,
	expos_exp_dt DATE,
	exposure_type STRING,
	expos_yr FLOAT,
	final_audit_value STRING,
	flood_deductible FLOAT,
	flood_limit FLOAT,
	id_for_data_loads STRING,
	insured_name STRING,
	last_updated_by STRING,
	last_updated_date DATE,
	load_notes STRING,
	location_number STRING,
	mac_id STRING,
	nbfu_fire_protection_class STRING,
	number_of_stories STRING,
	occupancy_type STRING,
	organization STRING,
	personal_stock FLOAT,
	pol_eff_dt DATE,
	pol_exp_dt DATE,
	policy_id_for_data_loads STRING,
	postal_code STRING,
	primary_coverage STRING,
	property_location_postal_code STRING,
	public_protection_class STRING,
	rc_record_id FLOAT,
	record_id_sort_order STRING,
	record_id STRING,
	risk_reference STRING,
	segment_product STRING,
	source_file_name STRING,
	policy_number STRING,
	square_footage FLOAT,
	insured_state STRING,
	stock FLOAT,
	subline_load STRING,
	time_element_limit FLOAT,
	unit_number FLOAT,
	unit_count FLOAT,
	unit_make STRING,
	unit_model STRING,
	unit_model_year FLOAT,
	unit_radius STRING,
	unit_territory STRING,
	unit_type STRING,
	vin STRING,
	value FLOAT,
	value_currency FLOAT,
	vehicle_class STRING,
	wind_method STRING,
	wind_hail_ded_amnt STRING,
	wind_hail_ded_pct STRING,
	wind_hail_exclusion STRING,
	year_built FLOAT,
	base_rate FLOAT,
	subline_code STRING,
	vehicle_base_zone STRING,
	vehicle_size STRING,
	vehicle_terminal_zone STRING,
	vehicle_use STRING,
	exposures FLOAT
);

CREATE OR REPLACE TABLE organization(
	as_of DATE,
	address_1 STRING,
	address_2 STRING,
	address_3 STRING,
	admin_unit_level_3_cd STRING,
	city STRING,
	country_iso_code STRING,
	description STRING,
	end_date DATE,
	fax_number STRING,
	last_updated_by STRING,
	last_updated_date DATE,
	latitude STRING,
	legal_name STRING,
	organization STRING,
	organization_status STRING,
	phone_number STRING,
	postal_code STRING,
	reason_for_leaving STRING,
	role STRING,
	start_date DATE,
	state STRING,
	update_geocoding STRING,
	organization_currency_iso_code STRING,
	org_last_updated_date_char STRING,
	organization_created_date_char STRING,
	organization_last_updated_date DATE,
	organization_created_date DATE
);

CREATE OR REPLACE TABLE policy(
	as_of DATE,
	book STRING,
	account_name STRING,
	specialty_program STRING,
	carrier STRING,
	LOB STRING,
	apd_premium FLOAT,
	axa_broker STRING,
	axa_other_mod_2 FLOAT,
	axa_other_mod_3 FLOAT,
	axa_other_mod_4 FLOAT,
	aggregate_limit FLOAT,
	annual_broker_fee FLOAT,
	annual_expenses FLOAT,
	annual_fees FLOAT,
	annual_fronting_charge FLOAT,
	annual_gross_total_premium FLOAT,
	annual_net_premium FLOAT,
	annual_other FLOAT,
	assigned_risk STRING,
	auto_liability_premium FLOAT,
	billing_code STRING,
	broker STRING,
	ca_territory_credit FLOAT,
	class_code_description STRING,
	comments STRING,
	commission_contribution STRING,
	created_by STRING,
	created_by_data_process STRING,
	policy_created_date DATE,
	created_date_rc_pol DATE,
	damage_premises FLOAT,
	deductible_limit_aggregate FLOAT,
	deductible_limit_amount FLOAT,
	deductible_limit_description STRING,
	department_of_transportation_num STRING,
	pol_eff_dt DATE,
	endorsements_broker_fee FLOAT,
	endorsements_expenses FLOAT,
	endorsements_fees FLOAT,
	endorsements_fronting_charge FLOAT,
	endorsements_gross_total_premium FLOAT,
	endorsements_net_premium FLOAT,
	endorsements_other FLOAT,
	equipment_premium FLOAT,
	expense_mod FLOAT,
	experience_mod FLOAT,
	pol_exp_dt DATE,
	extended_reporting_date DATE,
	fein STRING,
	uw_company STRING,
	gic_score FLOAT,
	governing_class_code STRING,
	id_for_loads STRING,
	insured_address_1 STRING,
	insured_address_2 STRING,
	insured_city STRING,
	insured_name STRING,
	insured_state STRING,
	insured_zip STRING,
	insurer_pct FLOAT,
	insurer_limit_aggregate FLOAT,
	insurer_limit_amount FLOAT,
	insurer_limit_description STRING,
	insurer_limit_excess_of FLOAT,
	is_this_tax STRING,
	last_updated_by STRING,
	policy_last_updated_date DATE,
	lead_insurer STRING,
	loss_ratio FLOAT,
	mac_id STRING,
	marketing_plan STRING,
	medical_expense STRING,
	model_year FLOAT,
	non_admitted_factor STRING,
	occurrence_limit STRING,
	organization STRING,
	account_id STRING,
	original_lob STRING,
	other_mod FLOAT,
	package_mod FLOAT,
	personal_advertising_injury STRING,
	policy_cancel_date STRING,
	policy_number STRING,
	policy_postal_code STRING,
	policy_renewal STRING,
	policy_state STRING,
	policy_transaction_type STRING,
	policy_trigger STRING,
	policy_year FLOAT,
	valen_score STRING,
	primary_coverage STRING,
	processing_layer FLOAT,
	product_code STRING,
	product_comp_works STRING,
	program_id STRING,
	program_name_link STRING,
	retro_date DATE,
	sir_limit_aggregate FLOAT,
	sir_limit_amount FLOAT,
	schedule_mod FLOAT,
	program STRING,
	sub_producer_address STRING,
	sub_producer_commission_char STRING,
	sub_producer STRING,
	sub_producer_number STRING,
	total_broker_fee FLOAT,
	total_expenses FLOAT,
	total_fees FLOAT,
	total_fronting_charge FLOAT,
	total_gross_total_premium FLOAT,
	total_net_premium FLOAT,
	total_other FLOAT,
	type STRING,
	xlia_score FLOAT,
	xls_score FLOAT,
	agent_name STRING,
	agent_state_code STRING,
	agent_zip_code STRING,
	aggregate_limit_premops FLOAT,
	aggregate_limit_products FLOAT,
	branch STRING,
	business_type_code STRING,
	cancel_flag STRING,
	consent_to_rate_flag STRING,
	employee_count_initial FLOAT,
	location_count_initial FLOAT,
	mod_final FLOAT,
	multiline_policy STRING,
	new_renew_flag FLOAT,
	policy_vehicles FLOAT,
	premium_mininum_flag STRING,
	quote_policy_term_number STRING,
	quote_term_effective_date DATE,
	rating_factor_experience_liab FLOAT,
	rating_factor_experience_pd FLOAT,
	rating_tier STRING,
	score_id STRING,
	sic_code STRING,
	term_termination_date DATE,
	underwriter STRING,
	written_premium_final FLOAT,
	written_premium_initial FLOAT,
	policy_currency_iso_code STRING,
	original_policy_inception_date DATE,
	sub_producer_commission FLOAT
);

CREATE OR REPLACE TABLE premium(
	as_of DATE,
	book STRING,
	account_name STRING,
	specialty_program STRING,
	carrier STRING,
	LOB STRING,
	al_premium FLOAT,
	al_or_apd_breakout STRING,
	apd_premium FLOAT,
	arborist_limit FLOAT,
	assigned_risk STRING,
	auto_liability_premium FLOAT,
	base_zone STRING,
	blanket_waiver STRING,
	broadened_cov_for_contractors STRING,
	building_deductible FLOAT,
	building_limit FLOAT,
	business_income_annual_premium FLOAT,
	business_income_deductible FLOAT,
	business_income_limit FLOAT,
	business_income_transaction_prem FLOAT,
	ca_territory_credit FLOAT,
	cat_coverage STRING,
	calculation_type_broker_fee STRING,
	calculation_type_fees STRING,
	calculation_type_fronting_charge STRING,
	calculation_type_other STRING,
	calculation_type_tax STRING,
	cancel_flag STRING,
	certified_acts_of_terrorism STRING,
	class_csp STRING,
	governing_class_code STRING,
	consent_to_rate STRING,
	construction_class STRING,
	construction_class_ii STRING,
	contact_email STRING,
	contact_name STRING,
	contact_phone_number STRING,
	contents_deductible FLOAT,
	contents_limit FLOAT,
	coverage STRING,
	created_by STRING,
	created_date DATE,
	created_date_rc_prem DATE,
	dot_number STRING,
	damage_to_rented_premises STRING,
	deductible FLOAT,
	ded_amnt_ded_1 FLOAT,
	ded_amnt_ded_14 FLOAT,
	ded_amnt_ded_15 FLOAT,
	ded_amnt_ded_16 FLOAT,
	ded_amnt_ded_2 FLOAT,
	ded_amnt_ded_3 FLOAT,
	ded_amnt_ded_4 FLOAT,
	ded_amnt_ded_5 FLOAT,
	ded_amnt_ded_6 FLOAT,
	ded_desc_ded_1 STRING,
	ded_desc_ded_14 STRING,
	ded_desc_ded_15 STRING,
	ded_desc_ded_16 STRING,
	ded_desc_ded_2 STRING,
	ded_desc_ded_3 STRING,
	ded_desc_ded_4 STRING,
	ded_desc_ded_5 STRING,
	ded_desc_ded_6 STRING,
	earthquake_ded FLOAT,
	expense_mod FLOAT,
	experience_mod FLOAT,
	fein STRING,
	firm STRING,
	class_code STRING,
	guaranteed_cost_flag STRING,
	id_used_for_data_loads STRING,
	insured_address_1 STRING,
	insured_address_2 STRING,
	insured_city STRING,
	insured_name STRING,
	insured_state STRING,
	insured_zip STRING,
	insurer STRING,
	is_renewal STRING,
	is_this_tax STRING,
	lcm FLOAT,
	landscape_or_arborist_services STRING,
	last_updated_by STRING,
	last_updated_date DATE,
	limit_amnt_limit_1 FLOAT,
	limit_amnt_limit_10 FLOAT,
	limit_amnt_limit_11 FLOAT,
	limit_amnt_limit_12 FLOAT,
	limit_amnt_limit_13 FLOAT,
	limit_amnt_limit_2 FLOAT,
	limit_amnt_limit_3 FLOAT,
	limit_amnt_limit_4 FLOAT,
	limit_amnt_limit_5 FLOAT,
	limit_amnt_limit_6 FLOAT,
	limit_amnt_limit_7 FLOAT,
	limit_amnt_limit_8 FLOAT,
	limit_amnt_limit_9 FLOAT,
	limit_desc_limit_1 STRING,
	limit_desc_limit_10 STRING,
	limit_desc_limit_11 STRING,
	limit_desc_limit_12 STRING,
	limit_desc_limit_13 STRING,
	limit_desc_limit_2 STRING,
	limit_desc_limit_3 STRING,
	limit_desc_limit_4 STRING,
	limit_desc_limit_5 STRING,
	limit_desc_limit_6 STRING,
	limit_desc_limit_7 STRING,
	limit_desc_limit_8 STRING,
	limit_desc_limit_9 STRING,
	load_notes STRING,
	loss_cost_original STRING,
	mod_sate STRING,
	model_description STRING,
	model_make STRING,
	model_year_char STRING,
	nbfu_fire_protection_class STRING,
	occupancy_type STRING,
	organization STRING,
	other_mod FLOAT,
	package_mod FLOAT,
	percentage_broker_fee FLOAT,
	percentage_fees FLOAT,
	percentage_fronting_charge FLOAT,
	percentage_other FLOAT,
	percentage_tax FLOAT,
	policy_cancel_date DATE,
	policy_id_for_load STRING,
	policy_number STRING,
	pol_eff_dt DATE,
	premium_transaction_type STRING,
	premium_vehicle_type STRING,
	primary_coverage STRING,
	radius_class STRING,
	raiting_tier STRING,
	rate FLOAT,
	ssu STRING,
	schedule_mod FLOAT,
	source_trans_type STRING,
	special_waiver STRING,
	stated_amount FLOAT,
	stop_gap_EL_cov STRING,
	stop_gap_limits FLOAT,
	subline_code STRING,
	subline_description STRING,
	terminal_zone STRING,
	territory STRING,
	time_element_limit FLOAT,
	total_broker_fee FLOAT,
	total_fees FLOAT,
	total_fronting_charge FLOAT,
	total_net_premium FLOAT,
	total_other FLOAT,
	total_paid FLOAT,
	total_tax FLOAT,
	unit_business_use STRING,
	unit_garage_zip STRING,
	unit_number_char STRING,
	vehicle_class STRING,
	vin STRING,
	wind_hail_deductible FLOAT,
	year_built FLOAT,
	coverage_code STRING,
	coverage_effective_date DATE,
	coverage_expiration_date DATE,
	coverage_symbol STRING,
	coverage_type_code STRING,
	deductible_amount FLOAT,
	limit_amount_final FLOAT,
	limit_amount_initial FLOAT,
	original_coverage_code STRING,
	original_coverage_desc STRING,
	premium_manual_final FLOAT,
	premium_manual_initial FLOAT,
	premium_written_final FLOAT,
	premium_written_initial FLOAT,
	rating_company_factor FLOAT,
	rating_tier_factor FLOAT,
	schedule_mod_factor FLOAT,
	territory_factor FLOAT,
	transaction_type STRING,
	unit_type_code STRING,
	premium_created_date_char STRING,
	linked_policy_id STRING,
	total_gross_total_premium FLOAT,
	total_outstanding FLOAT,
	model_year FLOAT,
	unit_number FLOAT,
	premium_created_date DATE
);

CREATE OR REPLACE TABLE ref_claims_patch(
	LOB STRING,
	Claim_Number BIGINT,
	Claimant_Sequence INT,
	Party_Update STRING,
	Coverage_Code_Detail STRING,
	Claim_Type STRING,
	ref_claim_number STRING
);

CREATE OR REPLACE TABLE ref_class_code(
	LOB STRING,
	class_code STRING,
	class_code_desc STRING,
	auto_size_class STRING,
	auto_size_class_type STRING,
	auto_size_class_weight STRING,
	auto_business_use_class STRING,
	auto_radius_class STRING,
	auto_fleet STRING,
	auto_classification STRING,
	auto_subclassification STRING,
	auto_type STRING,
	auto_subtype STRING
);

CREATE OR REPLACE TABLE ref_cov_desc(
	book STRING,
	LOB STRING,
	TPA STRING,
	descriptors_source_coverage STRING,
	cov_desc STRING,
	cov_desc_detail STRING
);

CREATE OR REPLACE TABLE ref_lob(
	coverage STRING,
	LOB STRING
);

CREATE OR REPLACE TABLE ref_marketing_plan(
	marketing_plan_code STRING,
	marketing_plan STRING
);

CREATE OR REPLACE TABLE ref_organization(
	organization STRING,
	book STRING,
	account_name STRING,
	specialty_program STRING,
	carrier STRING
);

CREATE OR REPLACE TABLE sp_premium_triangles(
	as_of DATE,
	book STRING,
	specialty_program STRING,
	carrier STRING,
	pol_eff_yr DECIMAL(11, 0),
	CAY SMALLINT,
	CAQ STRING,
	CAY_age FLOAT,
	LOB STRING,
	organization STRING,
	account_id STRING,
	mac_id STRING,
	insured_name STRING,
	policy_number STRING,
	uw_company STRING,
	broker STRING,
	axa_broker STRING,
	billing_code STRING,
	marketing_plan STRING,
	program STRING,
	pol_eff_dt DATE,
	pol_exp_dt DATE,
	multiline_policy STRING,
	new_renew_flag FLOAT,
	primary_coverage STRING,
	governing_class_code STRING,
	policy_state STRING,
	policy_state_region STRING,
	policy_state_ca_ind STRING,
	policy_zip STRING,
	insured_city STRING,
	insured_state STRING,
	insured_zip STRING,
	valen_score STRING,
	policy_count FLOAT,
	written_premium FLOAT,
	earned_premium FLOAT
);

CREATE OR REPLACE TABLE xref_marketing_plan_to_organization(
	marketing_plan_code STRING,
	marketing_plan STRING,
	organization STRING
);

In [0]:
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [claims](
	[as_of] [date] NULL,
	[book] [nvarchar](50) NULL,
	[account_name] [nvarchar](50) NULL,
	[specialty_program] [nvarchar](50) NULL,
	[carrier] [nvarchar](50) NULL,
	[LOB] [nvarchar](50) NULL,
	[accident_am_or_pm] [nvarchar](50) NULL,
	[accident_address_1] [nvarchar](50) NULL,
	[accident_address_2] [nvarchar](50) NULL,
	[accident_address_3] [nvarchar](50) NULL,
	[accident_city] [nvarchar](50) NULL,
	[accident_county] [nvarchar](50) NULL,
	[accident_department] [nvarchar](50) NULL,
	[accident_description] [char](2500) NULL,
	[accident_map] [nvarchar](200) NULL,
	[accident_postal_code] [nvarchar](50) NULL,
	[accident_shift] [nvarchar](50) NULL,
	[accident_state] [nvarchar](50) NULL,
	[accident_time] [nvarchar](50) NULL,
	[account_id] [nvarchar](50) NULL,
	[activity_before_loss] [nvarchar](50) NULL,
	[address_claimant_address_1] [nvarchar](50) NULL,
	[address_claimant_address_2] [nvarchar](50) NULL,
	[address_claimant_address_3] [nvarchar](50) NULL,
	[address_claimant_city] [nvarchar](50) NULL,
	[address_claimant_postal_code] [nvarchar](50) NULL,
	[address_claimant_state] [nvarchar](50) NULL,
	[alternate_claim_number] [nvarchar](50) NULL,
	[area_of_accident] [nvarchar](50) NULL,
	[attorney_representation_date] [date] NULL,
	[basel_business_line] [nvarchar](50) NULL,
	[basel_loss_event_type] [nvarchar](50) NULL,
	[benefit_state] [nvarchar](50) NULL,
	[bodily_injury] [nvarchar](50) NULL,
	[body_ncci] [nvarchar](50) NULL,
	[captive_os_expense] [float] NULL,
	[captive_os_indemnity_bi_pd_coll] [float] NULL,
	[captive_os_legal] [float] NULL,
	[captive_os_medical_comp] [float] NULL,
	[captive_os_other] [float] NULL,
	[captive_os_total] [float] NULL,
	[captive_paid_expense] [float] NULL,
	[captive_paid_indem_bi_pd_coll] [float] NULL,
	[captive_paid_legal] [float] NULL,
	[captive_paid_medical_comp] [float] NULL,
	[captive_paid_other] [float] NULL,
	[captive_paid_total] [float] NULL,
	[case_type] [nvarchar](50) NULL,
	[cause] [nvarchar](50) NULL,
	[cause_ncci] [nvarchar](50) NULL,
	[claim_deductible] [float] NULL,
	[claim_number] [nvarchar](50) NULL,
	[claim_or_incident] [nvarchar](50) NULL,
	[claim_stage] [nvarchar](50) NULL,
	[claim_status] [nvarchar](50) NULL,
	[claimant_death_date] [date] NULL,
	[claimant_first_name] [char](100) NULL,
	[claimant_last_name] [nvarchar](100) NULL,
	[claimant_m_i] [nvarchar](50) NULL,
	[claimant_name] [nvarchar](85) NULL,
	[claimant_title] [nvarchar](50) NULL,
	[claims_made_date] [date] NULL,
	[claim_created_date] [date] NULL,
	[claim_currency_iso_code] [nvarchar](50) NULL,
	[claim_last_updated_date_char] [nvarchar](50) NULL,
	[clash_claim] [nvarchar](50) NULL,
	[class_code] [nvarchar](50) NULL,
	[closed_date] [date] NULL,
	[construction_defect] [nvarchar](50) NULL,
	[controverted] [nvarchar](50) NULL,
	[coverage] [nvarchar](50) NULL,
	[coverage_code] [nvarchar](50) NULL,
	[created_by] [nvarchar](50) NULL,
	[created_by_data_process] [nvarchar](50) NULL,
	[created_date] [date] NULL,
	[created_date_rc_claim] [date] NULL,
	[TPA] [nvarchar](50) NULL,
	[date_suit_filed] [date] NULL,
	[date_placed_in_litigation] [date] NULL,
	[dates_source_claims_made_date] [date] NULL,
	[dates_source_close_date] [date] NULL,
	[dates_source_entry_date] [date] NULL,
	[dates_source_loss_date] [date] NULL,
	[dates_source_reopen_date] [date] NULL,
	[dates_source_report_date] [date] NULL,
	[days_paid] [float] NULL,
	[dde_copy_data_from_ee_record] [nvarchar](50) NULL,
	[ded_sir_os_expense] [float] NULL,
	[ded_sir_os_indemnity_bi_pd_coll] [float] NULL,
	[ded_sir_os_legal] [float] NULL,
	[ded_sir_os_medical_comp] [float] NULL,
	[ded_sir_os_other] [float] NULL,
	[ded_sir_os_total] [float] NULL,
	[ded_sir_paid_expense] [float] NULL,
	[ded_sir_paid_indem_bi_pd_coll] [float] NULL,
	[ded_sir_paid_legal] [float] NULL,
	[ded_sir_paid_medical_comp] [float] NULL,
	[ded_sir_paid_other] [float] NULL,
	[ded_sir_paid_total] [float] NULL,
	[deductible_description] [nvarchar](50) NULL,
	[deductible_from_policy] [nvarchar](50) NULL,
	[descr_amtrust_spoiled_claim_num] [nvarchar](50) NULL,
	[descriptors_source_body_part] [nvarchar](50) NULL,
	[descriptors_source_cause] [nvarchar](50) NULL,
	[descriptors_source_coverage] [nvarchar](50) NULL,
	[descriptors_source_location] [nvarchar](50) NULL,
	[descriptors_source_nature] [nvarchar](50) NULL,
	[descriptors_source_pol_eff_date] [date] NULL,
	[descriptors_source_policy_number] [nvarchar](50) NULL,
	[descriptors_source_policy_period] [nvarchar](50) NULL,
	[descriptors_source_state] [nvarchar](50) NULL,
	[descriptors_source_status] [nvarchar](50) NULL,
	[descriptors_travelers_sai_number] [nvarchar](50) NULL,
	[descr_travelers_st_paul_pol_num] [nvarchar](50) NULL,
	[downtime_units] [nvarchar](50) NULL,
	[driver_age] [float] NULL,
	[driver_birth_date] [date] NULL,
	[driver_first_name] [nvarchar](50) NULL,
	[driver_last_name] [nvarchar](50) NULL,
	[driver_m_i] [nvarchar](50) NULL,
	[driver_relationship] [nvarchar](50) NULL,
	[driver_sex] [nvarchar](50) NULL,
	[driver_title] [nvarchar](50) NULL,
	[drivers_license_number] [nvarchar](50) NULL,
	[driver_vehicle_additional_info] [nvarchar](50) NULL,
	[drivers_first_name] [nvarchar](50) NULL,
	[employer_notification_date] [date] NULL,
	[first_aid_care] [nvarchar](50) NULL,
	[garage_state] [nvarchar](50) NULL,
	[gross_incurred_expense] [float] NULL,
	[gross_incurred_indem_bi_pd_coll] [float] NULL,
	[gross_incurred_legal] [float] NULL,
	[gross_incurred_medical_comp] [float] NULL,
	[gross_incurred_other] [float] NULL,
	[gross_incurred_total] [float] NULL,
	[healthcare_address_1] [nvarchar](50) NULL,
	[healthcare_address_2] [nvarchar](50) NULL,
	[healthcare_city] [nvarchar](50) NULL,
	[healthcare_phone] [nvarchar](50) NULL,
	[healthcare_postal_code] [nvarchar](50) NULL,
	[healthcare_provider_type] [nvarchar](50) NULL,
	[healthcare_state] [nvarchar](50) NULL,
	[hire_car_required] [nvarchar](50) NULL,
	[hospitalized_overnight] [nvarchar](50) NULL,
	[id_for_data_loads] [nvarchar](100) NULL,
	[in_litigation] [nvarchar](50) NULL,
	[information_claimant_age] [float] NULL,
	[info_claimant_date_of_birth] [date] NULL,
	[information_claimant_gender] [nvarchar](50) NULL,
	[information_claimant_job_title] [nvarchar](50) NULL,
	[info_claimant_marital_status] [nvarchar](50) NULL,
	[info_claimant_phone_number] [nvarchar](50) NULL,
	[information_email] [nvarchar](50) NULL,
	[information_employee_id] [nvarchar](50) NULL,
	[information_employee_status] [nvarchar](50) NULL,
	[information_employment_end_date] [date] NULL,
	[info_employment_start_date] [date] NULL,
	[information_hire_date] [date] NULL,
	[information_job_termination_date] [date] NULL,
	[information_ncci_class_code] [nvarchar](50) NULL,
	[info_occupation_when_injured] [nvarchar](50) NULL,
	[information_supervisor_email] [nvarchar](50) NULL,
	[information_supervisor_name] [nvarchar](50) NULL,
	[information_tax_id] [nvarchar](50) NULL,
	[information_tax_id_admin] [nvarchar](50) NULL,
	[information_wage_amount] [float] NULL,
	[information_wage_type] [nvarchar](50) NULL,
	[initial_reserve] [float] NULL,
	[initial_treatment_date] [date] NULL,
	[injury_or_illness_description] [nvarchar](50) NULL,
	[insured_name] [nvarchar](150) NULL,
	[insured_driver_citation] [nvarchar](50) NULL,
	[insured_driver_information] [nvarchar](50) NULL,
	[insured_driver_injured] [nvarchar](50) NULL,
	[insured_driver_injury_desc] [nvarchar](50) NULL,
	[insured_vehicle_damage_desc] [nvarchar](50) NULL,
	[insured_vehicle_damaged] [nvarchar](50) NULL,
	[insurer_os_expense] [float] NULL,
	[insurer_os_indemnity_bi_pd_coll] [float] NULL,
	[insurer_os_legal] [float] NULL,
	[insurer_os_medical_comp] [float] NULL,
	[insurer_os_other] [float] NULL,
	[insurer_os_total] [float] NULL,
	[insurer_paid_expense] [float] NULL,
	[insurer_paid_indem_bi_pd_coll] [float] NULL,
	[insurer_paid_legal] [float] NULL,
	[insurer_paid_medical_comp] [float] NULL,
	[insurer_paid_other] [float] NULL,
	[insurer_paid_total] [float] NULL,
	[jif_incurred] [float] NULL,
	[jif_paid] [float] NULL,
	[last_financial_update] [date] NULL,
	[last_load_date] [date] NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[last_updated_date] [date] NULL,
	[last_worked_date] [date] NULL,
	[loss_date] [date] NULL,
	[loss_description_code] [char](100) NULL,
	[lost_days] [float] NULL,
	[lot_number] [nvarchar](50) NULL,
	[managed_care_fees] [nvarchar](50) NULL,
	[manufactured_date] [date] NULL,
	[medical_treatment_sought] [nvarchar](50) NULL,
	[model_number] [nvarchar](50) NULL,
	[nature] [nvarchar](50) NULL,
	[nature_ncci] [nvarchar](50) NULL,
	[net_incurred_expense] [float] NULL,
	[net_incurred_indem_bi_pd_coll] [float] NULL,
	[net_incurred_legal] [float] NULL,
	[net_incurred_medical_comp] [float] NULL,
	[net_incurred_other] [float] NULL,
	[net_incurred_total] [float] NULL,
	[object_substance_causing_harm] [nvarchar](50) NULL,
	[occ_severity_code] [nvarchar](50) NULL,
	[occurrence_lead_claim] [nvarchar](50) NULL,
	[event_number] [nvarchar](50) NULL,
	[office_code] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[osha_completed_by_rep] [nvarchar](50) NULL,
	[osha_injury_type] [nvarchar](50) NULL,
	[osha_privacy] [nvarchar](50) NULL,
	[osha_recordable] [nvarchar](50) NULL,
	[osha_rep_phone] [nvarchar](50) NULL,
	[osha_rep_title] [nvarchar](50) NULL,
	[part] [nvarchar](50) NULL,
	[physician_healthcare_name] [nvarchar](50) NULL,
	[police_professional_liability] [nvarchar](50) NULL,
	[pol_eff_dt] [date] NULL,
	[pol_exp_dt] [date] NULL,
	[policy_number] [nvarchar](50) NULL,
	[policy_record_id] [float] NULL,
	[policy_year] [float] NULL,
	[primary_insurer_limit] [nvarchar](50) NULL,
	[product_name] [nvarchar](50) NULL,
	[program_name] [nvarchar](100) NULL,
	[property_damage] [nvarchar](50) NULL,
	[purpose_of_journey] [nvarchar](50) NULL,
	[ql_case_type] [nvarchar](50) NULL,
	[recovery_expected_expense] [float] NULL,
	[recov_expected_indem_bi_pd_coll] [float] NULL,
	[recovery_expected_legal] [float] NULL,
	[recovery_expected_medical_comp] [float] NULL,
	[recovery_expected_other] [float] NULL,
	[recovery_expected_total] [float] NULL,
	[recovery_paid] [float] NULL,
	[recovery_received_expense] [float] NULL,
	[recov_received_indem_bi_pd_coll] [float] NULL,
	[recovery_received_legal] [float] NULL,
	[recovery_received_medical_comp] [float] NULL,
	[recovery_received_other] [float] NULL,
	[recovery_received_total] [float] NULL,
	[reimbursement] [nvarchar](50) NULL,
	[reimbursement_amount] [float] NULL,
	[reinsurance_recovery] [float] NULL,
	[release_to_work_date] [date] NULL,
	[reopen_date] [date] NULL,
	[reopened] [nvarchar](50) NULL,
	[report_date] [date] NULL,
	[report_lag] [float] NULL,
	[restricted_days] [float] NULL,
	[return_to_work_date] [date] NULL,
	[road_conditions] [nvarchar](50) NULL,
	[road_type] [nvarchar](50) NULL,
	[salvage_indicator] [nvarchar](50) NULL,
	[salvage_recovery] [float] NULL,
	[serial_number] [nvarchar](50) NULL,
	[subro_indicator] [nvarchar](50) NULL,
	[subro_recovery] [float] NULL,
	[suit_date] [date] NULL,
	[time_employee_began_work] [nvarchar](50) NULL,
	[total_incurred_net] [float] NULL,
	[total_loss] [nvarchar](50) NULL,
	[total_outstanding] [float] NULL,
	[total_paid] [float] NULL,
	[total_paid_net] [float] NULL,
	[total_recovery] [float] NULL,
	[town_incurred] [float] NULL,
	[town_paid] [float] NULL,
	[tpa_adjuster] [nvarchar](50) NULL,
	[treated_in_er] [nvarchar](50) NULL,
	[user_claim_type_override] [nvarchar](50) NULL,
	[vehicle_description] [nvarchar](50) NULL,
	[vehicle_downtime] [nvarchar](50) NULL,
	[vehicle_make] [nvarchar](50) NULL,
	[vehicle_model] [nvarchar](50) NULL,
	[vehicle_number] [nvarchar](50) NULL,
	[vehicle_tag] [nvarchar](50) NULL,
	[vehicle_year] [float] NULL,
	[vin] [nvarchar](50) NULL,
	[visibility] [nvarchar](50) NULL,
	[weather_conditions] [nvarchar](50) NULL,
	[work_began_am_or_pm] [nvarchar](50) NULL,
	[year] [float] NULL,
	[claim_last_updated_date] [date] NULL,
	[cov_desc] [varchar](50) NULL,
	[cov_desc_detail] [varchar](50) NULL,
	[lob_orig] [varchar](50) NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [claims_dh](
	[as_of] [date] NULL,
	[public_entity] [varchar](20) NULL,
	[member_name] [varchar](100) NULL,
	[claim_number] [varchar](30) NULL,
	[case_type] [varchar](50) NULL,
	[descriptors_source_policy_number] [varchar](50) NULL,
	[descriptors_source_policy_period] [varchar](20) NULL,
	[insured_name] [varchar](100) NULL,
	[policy_year] [smallint] NULL,
	[pol_eff_dt] [date] NULL,
	[pol_exp_dt] [date] NULL,
	[policy_number] [varchar](50) NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [claims_import_01](
	[accident_am_or_pm] [nvarchar](50) NULL,
	[accident_address_1] [nvarchar](50) NULL,
	[accident_address_2] [nvarchar](50) NULL,
	[accident_address_3] [nvarchar](50) NULL,
	[accident_city] [nvarchar](50) NULL,
	[accident_county] [nvarchar](50) NULL,
	[accident_department] [nvarchar](50) NULL,
	[accident_description] [char](2500) NULL,
	[accident_map] [nvarchar](200) NULL,
	[accident_postal_code] [nvarchar](50) NULL,
	[accident_shift] [nvarchar](50) NULL,
	[accident_state] [nvarchar](50) NULL,
	[accident_time] [nvarchar](50) NULL,
	[account_id] [nvarchar](50) NULL,
	[activity_before_loss] [nvarchar](50) NULL,
	[address_claimant_address_1] [nvarchar](50) NULL,
	[address_claimant_address_2] [nvarchar](50) NULL,
	[address_claimant_address_3] [nvarchar](50) NULL,
	[address_claimant_city] [nvarchar](50) NULL,
	[address_claimant_postal_code] [nvarchar](50) NULL,
	[address_claimant_state] [nvarchar](50) NULL,
	[alternate_claim_number] [nvarchar](50) NULL,
	[area_of_accident] [nvarchar](50) NULL,
	[attorney_representation_date] [date] NULL,
	[basel_business_line] [nvarchar](50) NULL,
	[basel_loss_event_type] [nvarchar](50) NULL,
	[benefit_state] [nvarchar](50) NULL,
	[bodily_injury] [nvarchar](50) NULL,
	[body_ncci] [nvarchar](50) NULL,
	[captive_os_expense] [float] NULL,
	[captive_os_indemnity_bi_pd_coll] [float] NULL,
	[captive_os_legal] [float] NULL,
	[captive_os_medical_comp] [float] NULL,
	[captive_os_other] [float] NULL,
	[captive_os_total] [float] NULL,
	[captive_paid_expense] [float] NULL,
	[captive_paid_indem_bi_pd_coll] [float] NULL,
	[captive_paid_legal] [float] NULL,
	[captive_paid_medical_comp] [float] NULL,
	[captive_paid_other] [float] NULL,
	[captive_paid_total] [float] NULL,
	[case_type] [nvarchar](50) NULL,
	[cause] [nvarchar](50) NULL,
	[cause_ncci] [nvarchar](50) NULL,
	[claim_deductible] [float] NULL,
	[claim_number] [nvarchar](50) NULL,
	[claim_or_incident] [nvarchar](50) NULL,
	[claim_stage] [nvarchar](50) NULL,
	[claim_status] [nvarchar](50) NULL,
	[claimant_death_date] [date] NULL,
	[claimant_first_name] [char](100) NULL,
	[claimant_last_name] [nvarchar](100) NULL,
	[claimant_m_i] [nvarchar](50) NULL,
	[claimant_name] [nvarchar](85) NULL,
	[claimant_title] [nvarchar](50) NULL,
	[claims_made_date] [date] NULL,
	[claim_created_date] [date] NULL,
	[claim_currency_iso_code] [nvarchar](50) NULL,
	[claim_last_updated_date_char] [nvarchar](50) NULL,
	[clash_claim] [nvarchar](50) NULL,
	[class_code] [nvarchar](50) NULL,
	[closed_date] [date] NULL,
	[construction_defect] [nvarchar](50) NULL,
	[controverted] [nvarchar](50) NULL,
	[coverage] [nvarchar](50) NULL,
	[coverage_code] [nvarchar](50) NULL,
	[created_by] [nvarchar](50) NULL,
	[created_by_data_process] [nvarchar](50) NULL,
	[created_date] [date] NULL,
	[created_date_rc_claim] [date] NULL,
	[TPA] [nvarchar](50) NULL,
	[date_suit_filed] [date] NULL,
	[date_placed_in_litigation] [date] NULL,
	[dates_source_claims_made_date] [date] NULL,
	[dates_source_close_date] [date] NULL,
	[dates_source_entry_date] [date] NULL,
	[dates_source_loss_date] [date] NULL,
	[dates_source_reopen_date] [date] NULL,
	[dates_source_report_date] [date] NULL,
	[days_paid] [float] NULL,
	[dde_copy_data_from_ee_record] [nvarchar](50) NULL,
	[ded_sir_os_expense] [float] NULL,
	[ded_sir_os_indemnity_bi_pd_coll] [float] NULL,
	[ded_sir_os_legal] [float] NULL,
	[ded_sir_os_medical_comp] [float] NULL,
	[ded_sir_os_other] [float] NULL,
	[ded_sir_os_total] [float] NULL,
	[ded_sir_paid_expense] [float] NULL,
	[ded_sir_paid_indem_bi_pd_coll] [float] NULL,
	[ded_sir_paid_legal] [float] NULL,
	[ded_sir_paid_medical_comp] [float] NULL,
	[ded_sir_paid_other] [float] NULL,
	[ded_sir_paid_total] [float] NULL,
	[deductible_description] [nvarchar](50) NULL,
	[deductible_from_policy] [nvarchar](50) NULL,
	[descr_amtrust_spoiled_claim_num] [nvarchar](50) NULL,
	[descriptors_source_body_part] [nvarchar](50) NULL,
	[descriptors_source_cause] [nvarchar](50) NULL,
	[descriptors_source_coverage] [nvarchar](50) NULL,
	[descriptors_source_location] [nvarchar](50) NULL,
	[descriptors_source_nature] [nvarchar](50) NULL,
	[descriptors_source_pol_eff_date] [date] NULL,
	[descriptors_source_policy_number] [nvarchar](50) NULL,
	[descriptors_source_policy_period] [nvarchar](50) NULL,
	[descriptors_source_state] [nvarchar](50) NULL,
	[descriptors_source_status] [nvarchar](50) NULL,
	[descriptors_travelers_sai_number] [nvarchar](50) NULL,
	[descr_travelers_st_paul_pol_num] [nvarchar](50) NULL,
	[downtime_units] [nvarchar](50) NULL,
	[driver_age] [float] NULL,
	[driver_birth_date] [date] NULL,
	[driver_first_name] [nvarchar](50) NULL,
	[driver_last_name] [nvarchar](50) NULL,
	[driver_m_i] [nvarchar](50) NULL,
	[driver_relationship] [nvarchar](50) NULL,
	[driver_sex] [nvarchar](50) NULL,
	[driver_title] [nvarchar](50) NULL,
	[drivers_license_number] [nvarchar](50) NULL,
	[driver_vehicle_additional_info] [nvarchar](50) NULL,
	[drivers_first_name] [nvarchar](50) NULL,
	[employer_notification_date] [date] NULL,
	[first_aid_care] [nvarchar](50) NULL,
	[garage_state] [nvarchar](50) NULL,
	[gross_incurred_expense] [float] NULL,
	[gross_incurred_indem_bi_pd_coll] [float] NULL,
	[gross_incurred_legal] [float] NULL,
	[gross_incurred_medical_comp] [float] NULL,
	[gross_incurred_other] [float] NULL,
	[gross_incurred_total] [float] NULL,
	[healthcare_address_1] [nvarchar](50) NULL,
	[healthcare_address_2] [nvarchar](50) NULL,
	[healthcare_city] [nvarchar](50) NULL,
	[healthcare_phone] [nvarchar](50) NULL,
	[healthcare_postal_code] [nvarchar](50) NULL,
	[healthcare_provider_type] [nvarchar](50) NULL,
	[healthcare_state] [nvarchar](50) NULL,
	[hire_car_required] [nvarchar](50) NULL,
	[hospitalized_overnight] [nvarchar](50) NULL,
	[id_for_data_loads] [nvarchar](100) NULL,
	[in_litigation] [nvarchar](50) NULL,
	[information_claimant_age] [float] NULL,
	[info_claimant_date_of_birth] [date] NULL,
	[information_claimant_gender] [nvarchar](50) NULL,
	[information_claimant_job_title] [nvarchar](50) NULL,
	[info_claimant_marital_status] [nvarchar](50) NULL,
	[info_claimant_phone_number] [nvarchar](50) NULL,
	[information_email] [nvarchar](50) NULL,
	[information_employee_id] [nvarchar](50) NULL,
	[information_employee_status] [nvarchar](50) NULL,
	[information_employment_end_date] [date] NULL,
	[info_employment_start_date] [date] NULL,
	[information_hire_date] [date] NULL,
	[information_job_termination_date] [date] NULL,
	[information_ncci_class_code] [nvarchar](50) NULL,
	[info_occupation_when_injured] [nvarchar](50) NULL,
	[information_supervisor_email] [nvarchar](50) NULL,
	[information_supervisor_name] [nvarchar](50) NULL,
	[information_tax_id] [nvarchar](50) NULL,
	[information_tax_id_admin] [nvarchar](50) NULL,
	[information_wage_amount] [float] NULL,
	[information_wage_type] [nvarchar](50) NULL,
	[initial_reserve] [float] NULL,
	[initial_treatment_date] [date] NULL,
	[injury_or_illness_description] [nvarchar](50) NULL,
	[insured_name] [nvarchar](150) NULL,
	[insured_driver_citation] [nvarchar](50) NULL,
	[insured_driver_information] [nvarchar](50) NULL,
	[insured_driver_injured] [nvarchar](50) NULL,
	[insured_driver_injury_desc] [nvarchar](50) NULL,
	[insured_vehicle_damage_desc] [nvarchar](50) NULL,
	[insured_vehicle_damaged] [nvarchar](50) NULL,
	[insurer_os_expense] [float] NULL,
	[insurer_os_indemnity_bi_pd_coll] [float] NULL,
	[insurer_os_legal] [float] NULL,
	[insurer_os_medical_comp] [float] NULL,
	[insurer_os_other] [float] NULL,
	[insurer_os_total] [float] NULL,
	[insurer_paid_expense] [float] NULL,
	[insurer_paid_indem_bi_pd_coll] [float] NULL,
	[insurer_paid_legal] [float] NULL,
	[insurer_paid_medical_comp] [float] NULL,
	[insurer_paid_other] [float] NULL,
	[insurer_paid_total] [float] NULL,
	[jif_incurred] [float] NULL,
	[jif_paid] [float] NULL,
	[last_financial_update] [date] NULL,
	[last_load_date] [date] NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[last_updated_date] [date] NULL,
	[last_worked_date] [date] NULL,
	[loss_date] [date] NULL,
	[loss_description_code] [char](100) NULL,
	[lost_days] [float] NULL,
	[lot_number] [nvarchar](50) NULL,
	[managed_care_fees] [nvarchar](50) NULL,
	[manufactured_date] [date] NULL,
	[medical_treatment_sought] [nvarchar](50) NULL,
	[model_number] [nvarchar](50) NULL,
	[nature] [nvarchar](50) NULL,
	[nature_ncci] [nvarchar](50) NULL,
	[net_incurred_expense] [float] NULL,
	[net_incurred_indem_bi_pd_coll] [float] NULL,
	[net_incurred_legal] [float] NULL,
	[net_incurred_medical_comp] [float] NULL,
	[net_incurred_other] [float] NULL,
	[net_incurred_total] [float] NULL,
	[object_substance_causing_harm] [nvarchar](50) NULL,
	[occ_severity_code] [nvarchar](50) NULL,
	[occurrence_lead_claim] [nvarchar](50) NULL,
	[event_number] [nvarchar](50) NULL,
	[office_code] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[osha_completed_by_rep] [nvarchar](50) NULL,
	[osha_injury_type] [nvarchar](50) NULL,
	[osha_privacy] [nvarchar](50) NULL,
	[osha_recordable] [nvarchar](50) NULL,
	[osha_rep_phone] [nvarchar](50) NULL,
	[osha_rep_title] [nvarchar](50) NULL,
	[part] [nvarchar](50) NULL,
	[physician_healthcare_name] [nvarchar](50) NULL,
	[police_professional_liability] [nvarchar](50) NULL,
	[pol_eff_dt] [date] NULL,
	[pol_exp_dt] [date] NULL,
	[policy_number] [nvarchar](50) NULL,
	[policy_record_id] [float] NULL,
	[policy_year] [float] NULL,
	[primary_insurer_limit] [nvarchar](50) NULL,
	[product_name] [nvarchar](50) NULL,
	[program_name] [nvarchar](100) NULL,
	[property_damage] [nvarchar](50) NULL,
	[purpose_of_journey] [nvarchar](50) NULL,
	[ql_case_type] [nvarchar](50) NULL,
	[recovery_expected_expense] [float] NULL,
	[recov_expected_indem_bi_pd_coll] [float] NULL,
	[recovery_expected_legal] [float] NULL,
	[recovery_expected_medical_comp] [float] NULL,
	[recovery_expected_other] [float] NULL,
	[recovery_expected_total] [float] NULL,
	[recovery_paid] [float] NULL,
	[recovery_received_expense] [float] NULL,
	[recov_received_indem_bi_pd_coll] [float] NULL,
	[recovery_received_legal] [float] NULL,
	[recovery_received_medical_comp] [float] NULL,
	[recovery_received_other] [float] NULL,
	[recovery_received_total] [float] NULL,
	[reimbursement] [nvarchar](50) NULL,
	[reimbursement_amount] [float] NULL,
	[reinsurance_recovery] [float] NULL,
	[release_to_work_date] [date] NULL,
	[reopen_date] [date] NULL,
	[reopened] [nvarchar](50) NULL,
	[report_date] [date] NULL,
	[report_lag] [float] NULL,
	[restricted_days] [float] NULL,
	[return_to_work_date] [date] NULL,
	[road_conditions] [nvarchar](50) NULL,
	[road_type] [nvarchar](50) NULL,
	[salvage_indicator] [nvarchar](50) NULL,
	[salvage_recovery] [float] NULL,
	[serial_number] [nvarchar](50) NULL,
	[subro_indicator] [nvarchar](50) NULL,
	[subro_recovery] [float] NULL,
	[suit_date] [date] NULL,
	[time_employee_began_work] [nvarchar](50) NULL,
	[total_incurred_net] [float] NULL,
	[total_loss] [nvarchar](50) NULL,
	[total_outstanding] [float] NULL,
	[total_paid] [float] NULL,
	[total_paid_net] [float] NULL,
	[total_recovery] [float] NULL,
	[town_incurred] [float] NULL,
	[town_paid] [float] NULL,
	[tpa_adjuster] [nvarchar](50) NULL,
	[treated_in_er] [nvarchar](50) NULL,
	[user_claim_type_override] [nvarchar](50) NULL,
	[vehicle_description] [nvarchar](50) NULL,
	[vehicle_downtime] [nvarchar](50) NULL,
	[vehicle_make] [nvarchar](50) NULL,
	[vehicle_model] [nvarchar](50) NULL,
	[vehicle_number] [nvarchar](50) NULL,
	[vehicle_tag] [nvarchar](50) NULL,
	[vehicle_year] [float] NULL,
	[vin] [nvarchar](50) NULL,
	[visibility] [nvarchar](50) NULL,
	[weather_conditions] [nvarchar](50) NULL,
	[work_began_am_or_pm] [nvarchar](50) NULL,
	[year] [float] NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [claims_import_02](
	[as_of] [date] NULL,
	[book] [nvarchar](50) NULL,
	[account_name] [nvarchar](50) NULL,
	[specialty_program] [nvarchar](50) NULL,
	[carrier] [nvarchar](50) NULL,
	[LOB] [nvarchar](50) NULL,
	[accident_am_or_pm] [nvarchar](50) NULL,
	[accident_address_1] [nvarchar](50) NULL,
	[accident_address_2] [nvarchar](50) NULL,
	[accident_address_3] [nvarchar](50) NULL,
	[accident_city] [nvarchar](50) NULL,
	[accident_county] [nvarchar](50) NULL,
	[accident_department] [nvarchar](50) NULL,
	[accident_description] [char](2500) NULL,
	[accident_map] [nvarchar](200) NULL,
	[accident_postal_code] [nvarchar](50) NULL,
	[accident_shift] [nvarchar](50) NULL,
	[accident_state] [nvarchar](50) NULL,
	[accident_time] [nvarchar](50) NULL,
	[account_id] [nvarchar](50) NULL,
	[activity_before_loss] [nvarchar](50) NULL,
	[address_claimant_address_1] [nvarchar](50) NULL,
	[address_claimant_address_2] [nvarchar](50) NULL,
	[address_claimant_address_3] [nvarchar](50) NULL,
	[address_claimant_city] [nvarchar](50) NULL,
	[address_claimant_postal_code] [nvarchar](50) NULL,
	[address_claimant_state] [nvarchar](50) NULL,
	[alternate_claim_number] [nvarchar](50) NULL,
	[area_of_accident] [nvarchar](50) NULL,
	[attorney_representation_date] [date] NULL,
	[basel_business_line] [nvarchar](50) NULL,
	[basel_loss_event_type] [nvarchar](50) NULL,
	[benefit_state] [nvarchar](50) NULL,
	[bodily_injury] [nvarchar](50) NULL,
	[body_ncci] [nvarchar](50) NULL,
	[captive_os_expense] [float] NULL,
	[captive_os_indemnity_bi_pd_coll] [float] NULL,
	[captive_os_legal] [float] NULL,
	[captive_os_medical_comp] [float] NULL,
	[captive_os_other] [float] NULL,
	[captive_os_total] [float] NULL,
	[captive_paid_expense] [float] NULL,
	[captive_paid_indem_bi_pd_coll] [float] NULL,
	[captive_paid_legal] [float] NULL,
	[captive_paid_medical_comp] [float] NULL,
	[captive_paid_other] [float] NULL,
	[captive_paid_total] [float] NULL,
	[case_type] [nvarchar](50) NULL,
	[cause] [nvarchar](50) NULL,
	[cause_ncci] [nvarchar](50) NULL,
	[claim_deductible] [float] NULL,
	[claim_number] [nvarchar](50) NULL,
	[claim_or_incident] [nvarchar](50) NULL,
	[claim_stage] [nvarchar](50) NULL,
	[claim_status] [nvarchar](50) NULL,
	[claimant_death_date] [date] NULL,
	[claimant_first_name] [char](100) NULL,
	[claimant_last_name] [nvarchar](100) NULL,
	[claimant_m_i] [nvarchar](50) NULL,
	[claimant_name] [nvarchar](85) NULL,
	[claimant_title] [nvarchar](50) NULL,
	[claims_made_date] [date] NULL,
	[claim_created_date] [date] NULL,
	[claim_currency_iso_code] [nvarchar](50) NULL,
	[claim_last_updated_date_char] [nvarchar](50) NULL,
	[clash_claim] [nvarchar](50) NULL,
	[class_code] [nvarchar](50) NULL,
	[closed_date] [date] NULL,
	[construction_defect] [nvarchar](50) NULL,
	[controverted] [nvarchar](50) NULL,
	[coverage] [nvarchar](50) NULL,
	[coverage_code] [nvarchar](50) NULL,
	[created_by] [nvarchar](50) NULL,
	[created_by_data_process] [nvarchar](50) NULL,
	[created_date] [date] NULL,
	[created_date_rc_claim] [date] NULL,
	[TPA] [nvarchar](50) NULL,
	[date_suit_filed] [date] NULL,
	[date_placed_in_litigation] [date] NULL,
	[dates_source_claims_made_date] [date] NULL,
	[dates_source_close_date] [date] NULL,
	[dates_source_entry_date] [date] NULL,
	[dates_source_loss_date] [date] NULL,
	[dates_source_reopen_date] [date] NULL,
	[dates_source_report_date] [date] NULL,
	[days_paid] [float] NULL,
	[dde_copy_data_from_ee_record] [nvarchar](50) NULL,
	[ded_sir_os_expense] [float] NULL,
	[ded_sir_os_indemnity_bi_pd_coll] [float] NULL,
	[ded_sir_os_legal] [float] NULL,
	[ded_sir_os_medical_comp] [float] NULL,
	[ded_sir_os_other] [float] NULL,
	[ded_sir_os_total] [float] NULL,
	[ded_sir_paid_expense] [float] NULL,
	[ded_sir_paid_indem_bi_pd_coll] [float] NULL,
	[ded_sir_paid_legal] [float] NULL,
	[ded_sir_paid_medical_comp] [float] NULL,
	[ded_sir_paid_other] [float] NULL,
	[ded_sir_paid_total] [float] NULL,
	[deductible_description] [nvarchar](50) NULL,
	[deductible_from_policy] [nvarchar](50) NULL,
	[descr_amtrust_spoiled_claim_num] [nvarchar](50) NULL,
	[descriptors_source_body_part] [nvarchar](50) NULL,
	[descriptors_source_cause] [nvarchar](50) NULL,
	[descriptors_source_coverage] [nvarchar](50) NULL,
	[descriptors_source_location] [nvarchar](50) NULL,
	[descriptors_source_nature] [nvarchar](50) NULL,
	[descriptors_source_pol_eff_date] [date] NULL,
	[descriptors_source_policy_number] [nvarchar](50) NULL,
	[descriptors_source_policy_period] [nvarchar](50) NULL,
	[descriptors_source_state] [nvarchar](50) NULL,
	[descriptors_source_status] [nvarchar](50) NULL,
	[descriptors_travelers_sai_number] [nvarchar](50) NULL,
	[descr_travelers_st_paul_pol_num] [nvarchar](50) NULL,
	[downtime_units] [nvarchar](50) NULL,
	[driver_age] [float] NULL,
	[driver_birth_date] [date] NULL,
	[driver_first_name] [nvarchar](50) NULL,
	[driver_last_name] [nvarchar](50) NULL,
	[driver_m_i] [nvarchar](50) NULL,
	[driver_relationship] [nvarchar](50) NULL,
	[driver_sex] [nvarchar](50) NULL,
	[driver_title] [nvarchar](50) NULL,
	[drivers_license_number] [nvarchar](50) NULL,
	[driver_vehicle_additional_info] [nvarchar](50) NULL,
	[drivers_first_name] [nvarchar](50) NULL,
	[employer_notification_date] [date] NULL,
	[first_aid_care] [nvarchar](50) NULL,
	[garage_state] [nvarchar](50) NULL,
	[gross_incurred_expense] [float] NULL,
	[gross_incurred_indem_bi_pd_coll] [float] NULL,
	[gross_incurred_legal] [float] NULL,
	[gross_incurred_medical_comp] [float] NULL,
	[gross_incurred_other] [float] NULL,
	[gross_incurred_total] [float] NULL,
	[healthcare_address_1] [nvarchar](50) NULL,
	[healthcare_address_2] [nvarchar](50) NULL,
	[healthcare_city] [nvarchar](50) NULL,
	[healthcare_phone] [nvarchar](50) NULL,
	[healthcare_postal_code] [nvarchar](50) NULL,
	[healthcare_provider_type] [nvarchar](50) NULL,
	[healthcare_state] [nvarchar](50) NULL,
	[hire_car_required] [nvarchar](50) NULL,
	[hospitalized_overnight] [nvarchar](50) NULL,
	[id_for_data_loads] [nvarchar](100) NULL,
	[in_litigation] [nvarchar](50) NULL,
	[information_claimant_age] [float] NULL,
	[info_claimant_date_of_birth] [date] NULL,
	[information_claimant_gender] [nvarchar](50) NULL,
	[information_claimant_job_title] [nvarchar](50) NULL,
	[info_claimant_marital_status] [nvarchar](50) NULL,
	[info_claimant_phone_number] [nvarchar](50) NULL,
	[information_email] [nvarchar](50) NULL,
	[information_employee_id] [nvarchar](50) NULL,
	[information_employee_status] [nvarchar](50) NULL,
	[information_employment_end_date] [date] NULL,
	[info_employment_start_date] [date] NULL,
	[information_hire_date] [date] NULL,
	[information_job_termination_date] [date] NULL,
	[information_ncci_class_code] [nvarchar](50) NULL,
	[info_occupation_when_injured] [nvarchar](50) NULL,
	[information_supervisor_email] [nvarchar](50) NULL,
	[information_supervisor_name] [nvarchar](50) NULL,
	[information_tax_id] [nvarchar](50) NULL,
	[information_tax_id_admin] [nvarchar](50) NULL,
	[information_wage_amount] [float] NULL,
	[information_wage_type] [nvarchar](50) NULL,
	[initial_reserve] [float] NULL,
	[initial_treatment_date] [date] NULL,
	[injury_or_illness_description] [nvarchar](50) NULL,
	[insured_name] [nvarchar](150) NULL,
	[insured_driver_citation] [nvarchar](50) NULL,
	[insured_driver_information] [nvarchar](50) NULL,
	[insured_driver_injured] [nvarchar](50) NULL,
	[insured_driver_injury_desc] [nvarchar](50) NULL,
	[insured_vehicle_damage_desc] [nvarchar](50) NULL,
	[insured_vehicle_damaged] [nvarchar](50) NULL,
	[insurer_os_expense] [float] NULL,
	[insurer_os_indemnity_bi_pd_coll] [float] NULL,
	[insurer_os_legal] [float] NULL,
	[insurer_os_medical_comp] [float] NULL,
	[insurer_os_other] [float] NULL,
	[insurer_os_total] [float] NULL,
	[insurer_paid_expense] [float] NULL,
	[insurer_paid_indem_bi_pd_coll] [float] NULL,
	[insurer_paid_legal] [float] NULL,
	[insurer_paid_medical_comp] [float] NULL,
	[insurer_paid_other] [float] NULL,
	[insurer_paid_total] [float] NULL,
	[jif_incurred] [float] NULL,
	[jif_paid] [float] NULL,
	[last_financial_update] [date] NULL,
	[last_load_date] [date] NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[last_updated_date] [date] NULL,
	[last_worked_date] [date] NULL,
	[loss_date] [date] NULL,
	[loss_description_code] [char](100) NULL,
	[lost_days] [float] NULL,
	[lot_number] [nvarchar](50) NULL,
	[managed_care_fees] [nvarchar](50) NULL,
	[manufactured_date] [date] NULL,
	[medical_treatment_sought] [nvarchar](50) NULL,
	[model_number] [nvarchar](50) NULL,
	[nature] [nvarchar](50) NULL,
	[nature_ncci] [nvarchar](50) NULL,
	[net_incurred_expense] [float] NULL,
	[net_incurred_indem_bi_pd_coll] [float] NULL,
	[net_incurred_legal] [float] NULL,
	[net_incurred_medical_comp] [float] NULL,
	[net_incurred_other] [float] NULL,
	[net_incurred_total] [float] NULL,
	[object_substance_causing_harm] [nvarchar](50) NULL,
	[occ_severity_code] [nvarchar](50) NULL,
	[occurrence_lead_claim] [nvarchar](50) NULL,
	[event_number] [nvarchar](50) NULL,
	[office_code] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[osha_completed_by_rep] [nvarchar](50) NULL,
	[osha_injury_type] [nvarchar](50) NULL,
	[osha_privacy] [nvarchar](50) NULL,
	[osha_recordable] [nvarchar](50) NULL,
	[osha_rep_phone] [nvarchar](50) NULL,
	[osha_rep_title] [nvarchar](50) NULL,
	[part] [nvarchar](50) NULL,
	[physician_healthcare_name] [nvarchar](50) NULL,
	[police_professional_liability] [nvarchar](50) NULL,
	[pol_eff_dt] [date] NULL,
	[pol_exp_dt] [date] NULL,
	[policy_number] [nvarchar](50) NULL,
	[policy_record_id] [float] NULL,
	[policy_year] [float] NULL,
	[primary_insurer_limit] [nvarchar](50) NULL,
	[product_name] [nvarchar](50) NULL,
	[program_name] [nvarchar](100) NULL,
	[property_damage] [nvarchar](50) NULL,
	[purpose_of_journey] [nvarchar](50) NULL,
	[ql_case_type] [nvarchar](50) NULL,
	[recovery_expected_expense] [float] NULL,
	[recov_expected_indem_bi_pd_coll] [float] NULL,
	[recovery_expected_legal] [float] NULL,
	[recovery_expected_medical_comp] [float] NULL,
	[recovery_expected_other] [float] NULL,
	[recovery_expected_total] [float] NULL,
	[recovery_paid] [float] NULL,
	[recovery_received_expense] [float] NULL,
	[recov_received_indem_bi_pd_coll] [float] NULL,
	[recovery_received_legal] [float] NULL,
	[recovery_received_medical_comp] [float] NULL,
	[recovery_received_other] [float] NULL,
	[recovery_received_total] [float] NULL,
	[reimbursement] [nvarchar](50) NULL,
	[reimbursement_amount] [float] NULL,
	[reinsurance_recovery] [float] NULL,
	[release_to_work_date] [date] NULL,
	[reopen_date] [date] NULL,
	[reopened] [nvarchar](50) NULL,
	[report_date] [date] NULL,
	[report_lag] [float] NULL,
	[restricted_days] [float] NULL,
	[return_to_work_date] [date] NULL,
	[road_conditions] [nvarchar](50) NULL,
	[road_type] [nvarchar](50) NULL,
	[salvage_indicator] [nvarchar](50) NULL,
	[salvage_recovery] [float] NULL,
	[serial_number] [nvarchar](50) NULL,
	[subro_indicator] [nvarchar](50) NULL,
	[subro_recovery] [float] NULL,
	[suit_date] [date] NULL,
	[time_employee_began_work] [nvarchar](50) NULL,
	[total_incurred_net] [float] NULL,
	[total_loss] [nvarchar](50) NULL,
	[total_outstanding] [float] NULL,
	[total_paid] [float] NULL,
	[total_paid_net] [float] NULL,
	[total_recovery] [float] NULL,
	[town_incurred] [float] NULL,
	[town_paid] [float] NULL,
	[tpa_adjuster] [nvarchar](50) NULL,
	[treated_in_er] [nvarchar](50) NULL,
	[user_claim_type_override] [nvarchar](50) NULL,
	[vehicle_description] [nvarchar](50) NULL,
	[vehicle_downtime] [nvarchar](50) NULL,
	[vehicle_make] [nvarchar](50) NULL,
	[vehicle_model] [nvarchar](50) NULL,
	[vehicle_number] [nvarchar](50) NULL,
	[vehicle_tag] [nvarchar](50) NULL,
	[vehicle_year] [float] NULL,
	[vin] [nvarchar](50) NULL,
	[visibility] [nvarchar](50) NULL,
	[weather_conditions] [nvarchar](50) NULL,
	[work_began_am_or_pm] [nvarchar](50) NULL,
	[year] [float] NULL,
	[claim_last_updated_date] [date] NULL,
	[cov_desc] [nvarchar](50) NULL,
	[cov_desc_detail] [nvarchar](50) NULL,
	[lob_orig] [varchar](50) NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [claims_qualynx](
	[as_of] [date] NULL,
	[public_entity] [varchar](20) NULL,
	[member_name] [varchar](100) NULL,
	[policy_number] [varchar](50) NULL,
	[descriptors_source_policy_number] [varchar](50) NULL,
	[insured_name] [varchar](100) NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [exposures](
	[as_of] [date] NULL,
	[book] [nvarchar](50) NULL,
	[account_name] [nvarchar](50) NULL,
	[specialty_program] [nvarchar](50) NULL,
	[carrier] [nvarchar](50) NULL,
	[LOB] [nvarchar](50) NULL,
	[aop_deductible] [nvarchar](50) NULL,
	[address_1] [char](100) NULL,
	[address_2] [char](100) NULL,
	[building_limit] [float] NULL,
	[building_number] [nvarchar](50) NULL,
	[city] [nvarchar](50) NULL,
	[class_code] [nvarchar](50) NULL,
	[construction] [nvarchar](50) NULL,
	[construction_class] [nvarchar](50) NULL,
	[contents_limit] [float] NULL,
	[coverage] [nvarchar](50) NULL,
	[coverage_reference] [nvarchar](50) NULL,
	[created_by] [nvarchar](50) NULL,
	[created_by_data_process] [nvarchar](50) NULL,
	[exposure_created_date] [date] NULL,
	[created_date_rc_exp] [date] NULL,
	[deductible] [float] NULL,
	[deductible_type] [nvarchar](50) NULL,
	[eq_deductible] [float] NULL,
	[eq_limit] [float] NULL,
	[endorsement_number] [float] NULL,
	[expos_dt] [date] NULL,
	[expos_eff_dt] [date] NULL,
	[expos_exp_dt] [date] NULL,
	[exposure_type] [nvarchar](50) NULL,
	[expos_yr] [float] NULL,
	[final_audit_value] [nvarchar](50) NULL,
	[flood_deductible] [float] NULL,
	[flood_limit] [float] NULL,
	[id_for_data_loads] [char](200) NULL,
	[insured_name] [nvarchar](150) NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[last_updated_date] [date] NULL,
	[load_notes] [nvarchar](50) NULL,
	[location_number] [nvarchar](50) NULL,
	[mac_id] [nvarchar](50) NULL,
	[nbfu_fire_protection_class] [nvarchar](50) NULL,
	[number_of_stories] [nvarchar](50) NULL,
	[occupancy_type] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[personal_stock] [float] NULL,
	[pol_eff_dt] [date] NULL,
	[pol_exp_dt] [date] NULL,
	[policy_id_for_data_loads] [nvarchar](100) NULL,
	[postal_code] [char](75) NULL,
	[primary_coverage] [nvarchar](50) NULL,
	[property_location_postal_code] [nvarchar](50) NULL,
	[public_protection_class] [nvarchar](50) NULL,
	[rc_record_id] [float] NULL,
	[record_id_sort_order] [nvarchar](50) NULL,
	[record_id] [nvarchar](50) NULL,
	[risk_reference] [nvarchar](50) NULL,
	[segment_product] [nvarchar](50) NULL,
	[source_file_name] [nvarchar](50) NULL,
	[policy_number] [nvarchar](50) NULL,
	[square_footage] [float] NULL,
	[insured_state] [nvarchar](50) NULL,
	[stock] [float] NULL,
	[subline_load] [nvarchar](50) NULL,
	[time_element_limit] [float] NULL,
	[unit_number] [float] NULL,
	[unit_count] [float] NULL,
	[unit_make] [nvarchar](50) NULL,
	[unit_model] [nvarchar](50) NULL,
	[unit_model_year] [float] NULL,
	[unit_radius] [nvarchar](50) NULL,
	[unit_territory] [nvarchar](50) NULL,
	[unit_type] [nvarchar](50) NULL,
	[vin] [nvarchar](50) NULL,
	[value] [float] NULL,
	[value_currency] [float] NULL,
	[vehicle_class] [nvarchar](50) NULL,
	[wind_method] [nvarchar](50) NULL,
	[wind_hail_ded_amnt] [nvarchar](50) NULL,
	[wind_hail_ded_pct] [nvarchar](50) NULL,
	[wind_hail_exclusion] [nvarchar](50) NULL,
	[year_built] [float] NULL,
	[base_rate] [float] NULL,
	[subline_code] [nvarchar](50) NULL,
	[vehicle_base_zone] [nvarchar](50) NULL,
	[vehicle_size] [nvarchar](50) NULL,
	[vehicle_terminal_zone] [nvarchar](50) NULL,
	[vehicle_use] [nvarchar](50) NULL,
	[exposures] [float] NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [exposures_import_01](
	[aop_deductible] [nvarchar](50) NULL,
	[address_1] [char](100) NULL,
	[address_2] [char](100) NULL,
	[building_limit] [float] NULL,
	[building_number] [nvarchar](50) NULL,
	[city] [nvarchar](50) NULL,
	[class_code] [nvarchar](50) NULL,
	[construction] [nvarchar](50) NULL,
	[construction_class] [nvarchar](50) NULL,
	[contents_limit] [float] NULL,
	[coverage] [nvarchar](50) NULL,
	[coverage_reference] [nvarchar](50) NULL,
	[created_by] [nvarchar](50) NULL,
	[created_by_data_process] [nvarchar](50) NULL,
	[exposure_created_date] [date] NULL,
	[created_date_rc_exp] [date] NULL,
	[deductible] [float] NULL,
	[deductible_type] [nvarchar](50) NULL,
	[eq_deductible] [float] NULL,
	[eq_limit] [float] NULL,
	[endorsement_number] [float] NULL,
	[expos_dt] [date] NULL,
	[expos_eff_dt] [date] NULL,
	[expos_exp_dt] [date] NULL,
	[exposure_type] [nvarchar](50) NULL,
	[expos_yr] [float] NULL,
	[final_audit_value] [nvarchar](50) NULL,
	[flood_deductible] [float] NULL,
	[flood_limit] [float] NULL,
	[id_for_data_loads] [char](200) NULL,
	[insured_name] [nvarchar](150) NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[last_updated_date] [date] NULL,
	[load_notes] [nvarchar](50) NULL,
	[location_number] [nvarchar](50) NULL,
	[mac_id] [nvarchar](50) NULL,
	[nbfu_fire_protection_class] [nvarchar](50) NULL,
	[number_of_stories] [nvarchar](50) NULL,
	[occupancy_type] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[personal_stock] [float] NULL,
	[pol_eff_dt] [date] NULL,
	[pol_exp_dt] [date] NULL,
	[policy_id_for_data_loads] [nvarchar](100) NULL,
	[postal_code] [char](75) NULL,
	[primary_coverage] [nvarchar](50) NULL,
	[property_location_postal_code] [nvarchar](50) NULL,
	[public_protection_class] [nvarchar](50) NULL,
	[rc_record_id] [float] NULL,
	[record_id_sort_order] [nvarchar](50) NULL,
	[record_id] [nvarchar](50) NULL,
	[risk_reference] [nvarchar](50) NULL,
	[segment_product] [nvarchar](50) NULL,
	[source_file_name] [nvarchar](50) NULL,
	[policy_number] [nvarchar](50) NULL,
	[square_footage] [float] NULL,
	[insured_state] [nvarchar](50) NULL,
	[stock] [float] NULL,
	[subline_load] [nvarchar](50) NULL,
	[time_element_limit] [float] NULL,
	[unit_number] [float] NULL,
	[unit_count] [float] NULL,
	[unit_make] [nvarchar](50) NULL,
	[unit_model] [nvarchar](50) NULL,
	[unit_model_year] [float] NULL,
	[unit_radius] [nvarchar](50) NULL,
	[unit_territory] [nvarchar](50) NULL,
	[unit_type] [nvarchar](50) NULL,
	[vin] [nvarchar](50) NULL,
	[value] [float] NULL,
	[value_currency] [float] NULL,
	[vehicle_class] [nvarchar](50) NULL,
	[wind_method] [nvarchar](50) NULL,
	[wind_hail_ded_amnt] [nvarchar](50) NULL,
	[wind_hail_ded_pct] [nvarchar](50) NULL,
	[wind_hail_exclusion] [nvarchar](50) NULL,
	[year_built] [float] NULL,
	[base_rate] [float] NULL,
	[subline_code] [nvarchar](50) NULL,
	[vehicle_base_zone] [nvarchar](50) NULL,
	[vehicle_size] [nvarchar](50) NULL,
	[vehicle_terminal_zone] [nvarchar](50) NULL,
	[vehicle_use] [nvarchar](50) NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [exposures_import_02](
	[as_of] [date] NULL,
	[book] [nvarchar](50) NULL,
	[account_name] [nvarchar](50) NULL,
	[specialty_program] [nvarchar](50) NULL,
	[carrier] [nvarchar](50) NULL,
	[LOB] [nvarchar](50) NULL,
	[aop_deductible] [nvarchar](50) NULL,
	[address_1] [char](100) NULL,
	[address_2] [char](100) NULL,
	[building_limit] [float] NULL,
	[building_number] [nvarchar](50) NULL,
	[city] [nvarchar](50) NULL,
	[class_code] [nvarchar](50) NULL,
	[construction] [nvarchar](50) NULL,
	[construction_class] [nvarchar](50) NULL,
	[contents_limit] [float] NULL,
	[coverage] [nvarchar](50) NULL,
	[coverage_reference] [nvarchar](50) NULL,
	[created_by] [nvarchar](50) NULL,
	[created_by_data_process] [nvarchar](50) NULL,
	[exposure_created_date] [date] NULL,
	[created_date_rc_exp] [date] NULL,
	[deductible] [float] NULL,
	[deductible_type] [nvarchar](50) NULL,
	[eq_deductible] [float] NULL,
	[eq_limit] [float] NULL,
	[endorsement_number] [float] NULL,
	[expos_dt] [date] NULL,
	[expos_eff_dt] [date] NULL,
	[expos_exp_dt] [date] NULL,
	[exposure_type] [nvarchar](50) NULL,
	[expos_yr] [float] NULL,
	[final_audit_value] [nvarchar](50) NULL,
	[flood_deductible] [float] NULL,
	[flood_limit] [float] NULL,
	[id_for_data_loads] [char](200) NULL,
	[insured_name] [nvarchar](150) NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[last_updated_date] [date] NULL,
	[load_notes] [nvarchar](50) NULL,
	[location_number] [nvarchar](50) NULL,
	[mac_id] [nvarchar](50) NULL,
	[nbfu_fire_protection_class] [nvarchar](50) NULL,
	[number_of_stories] [nvarchar](50) NULL,
	[occupancy_type] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[personal_stock] [float] NULL,
	[pol_eff_dt] [date] NULL,
	[pol_exp_dt] [date] NULL,
	[policy_id_for_data_loads] [nvarchar](100) NULL,
	[postal_code] [char](75) NULL,
	[primary_coverage] [nvarchar](50) NULL,
	[property_location_postal_code] [nvarchar](50) NULL,
	[public_protection_class] [nvarchar](50) NULL,
	[rc_record_id] [float] NULL,
	[record_id_sort_order] [nvarchar](50) NULL,
	[record_id] [nvarchar](50) NULL,
	[risk_reference] [nvarchar](50) NULL,
	[segment_product] [nvarchar](50) NULL,
	[source_file_name] [nvarchar](50) NULL,
	[policy_number] [nvarchar](50) NULL,
	[square_footage] [float] NULL,
	[insured_state] [nvarchar](50) NULL,
	[stock] [float] NULL,
	[subline_load] [nvarchar](50) NULL,
	[time_element_limit] [float] NULL,
	[unit_number] [float] NULL,
	[unit_count] [float] NULL,
	[unit_make] [nvarchar](50) NULL,
	[unit_model] [nvarchar](50) NULL,
	[unit_model_year] [float] NULL,
	[unit_radius] [nvarchar](50) NULL,
	[unit_territory] [nvarchar](50) NULL,
	[unit_type] [nvarchar](50) NULL,
	[vin] [nvarchar](50) NULL,
	[value] [float] NULL,
	[value_currency] [float] NULL,
	[vehicle_class] [nvarchar](50) NULL,
	[wind_method] [nvarchar](50) NULL,
	[wind_hail_ded_amnt] [nvarchar](50) NULL,
	[wind_hail_ded_pct] [nvarchar](50) NULL,
	[wind_hail_exclusion] [nvarchar](50) NULL,
	[year_built] [float] NULL,
	[base_rate] [float] NULL,
	[subline_code] [nvarchar](50) NULL,
	[vehicle_base_zone] [nvarchar](50) NULL,
	[vehicle_size] [nvarchar](50) NULL,
	[vehicle_terminal_zone] [nvarchar](50) NULL,
	[vehicle_use] [nvarchar](50) NULL,
	[exposures] [float] NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [organization](
	[as_of] [date] NULL,
	[address_1] [nvarchar](50) NULL,
	[address_2] [nvarchar](50) NULL,
	[address_3] [nvarchar](50) NULL,
	[admin_unit_level_3_cd] [nvarchar](50) NULL,
	[city] [nvarchar](50) NULL,
	[country_iso_code] [nvarchar](50) NULL,
	[description] [nvarchar](50) NULL,
	[end_date] [date] NULL,
	[fax_number] [nvarchar](50) NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[last_updated_date] [date] NULL,
	[latitude] [nvarchar](50) NULL,
	[legal_name] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[organization_status] [nvarchar](50) NULL,
	[phone_number] [nvarchar](50) NULL,
	[postal_code] [nvarchar](50) NULL,
	[reason_for_leaving] [nvarchar](50) NULL,
	[role] [nvarchar](50) NULL,
	[start_date] [date] NULL,
	[state] [nvarchar](50) NULL,
	[update_geocoding] [nvarchar](50) NULL,
	[organization_currency_iso_code] [nvarchar](50) NULL,
	[org_last_updated_date_char] [nvarchar](50) NULL,
	[organization_created_date_char] [nvarchar](50) NULL,
	[organization_last_updated_date] [date] NULL,
	[organization_created_date] [date] NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [organization_import_01](
	[address_1] [nvarchar](50) NULL,
	[address_2] [nvarchar](50) NULL,
	[address_3] [nvarchar](50) NULL,
	[admin_unit_level_3_cd] [nvarchar](50) NULL,
	[city] [nvarchar](50) NULL,
	[country_iso_code] [nvarchar](50) NULL,
	[description] [nvarchar](50) NULL,
	[end_date] [date] NULL,
	[fax_number] [nvarchar](50) NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[last_updated_date] [date] NULL,
	[latitude] [nvarchar](50) NULL,
	[legal_name] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[organization_status] [nvarchar](50) NULL,
	[phone_number] [nvarchar](50) NULL,
	[postal_code] [nvarchar](50) NULL,
	[reason_for_leaving] [nvarchar](50) NULL,
	[role] [nvarchar](50) NULL,
	[start_date] [date] NULL,
	[state] [nvarchar](50) NULL,
	[update_geocoding] [nvarchar](50) NULL,
	[organization_currency_iso_code] [nvarchar](50) NULL,
	[org_last_updated_date_char] [nvarchar](50) NULL,
	[organization_created_date_char] [nvarchar](50) NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [organization_import_02](
	[as_of] [date] NULL,
	[address_1] [nvarchar](50) NULL,
	[address_2] [nvarchar](50) NULL,
	[address_3] [nvarchar](50) NULL,
	[admin_unit_level_3_cd] [nvarchar](50) NULL,
	[city] [nvarchar](50) NULL,
	[country_iso_code] [nvarchar](50) NULL,
	[description] [nvarchar](50) NULL,
	[end_date] [date] NULL,
	[fax_number] [nvarchar](50) NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[last_updated_date] [date] NULL,
	[latitude] [nvarchar](50) NULL,
	[legal_name] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[organization_status] [nvarchar](50) NULL,
	[phone_number] [nvarchar](50) NULL,
	[postal_code] [nvarchar](50) NULL,
	[reason_for_leaving] [nvarchar](50) NULL,
	[role] [nvarchar](50) NULL,
	[start_date] [date] NULL,
	[state] [nvarchar](50) NULL,
	[update_geocoding] [nvarchar](50) NULL,
	[organization_currency_iso_code] [nvarchar](50) NULL,
	[org_last_updated_date_char] [nvarchar](50) NULL,
	[organization_created_date_char] [nvarchar](50) NULL,
	[organization_last_updated_date] [date] NULL,
	[organization_created_date] [date] NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [policy](
	[as_of] [date] NULL,
	[book] [nvarchar](50) NULL,
	[account_name] [nvarchar](50) NULL,
	[specialty_program] [nvarchar](50) NULL,
	[carrier] [nvarchar](50) NULL,
	[LOB] [nvarchar](50) NULL,
	[apd_premium] [float] NULL,
	[axa_broker] [nvarchar](50) NULL,
	[axa_other_mod_2] [float] NULL,
	[axa_other_mod_3] [float] NULL,
	[axa_other_mod_4] [float] NULL,
	[aggregate_limit] [float] NULL,
	[annual_broker_fee] [float] NULL,
	[annual_expenses] [float] NULL,
	[annual_fees] [float] NULL,
	[annual_fronting_charge] [float] NULL,
	[annual_gross_total_premium] [float] NULL,
	[annual_net_premium] [float] NULL,
	[annual_other] [float] NULL,
	[assigned_risk] [nvarchar](50) NULL,
	[auto_liability_premium] [float] NULL,
	[billing_code] [nvarchar](50) NULL,
	[broker] [nvarchar](50) NULL,
	[ca_territory_credit] [float] NULL,
	[class_code_description] [char](200) NULL,
	[comments] [char](4000) NULL,
	[commission_contribution] [nvarchar](50) NULL,
	[created_by] [nvarchar](50) NULL,
	[created_by_data_process] [nvarchar](50) NULL,
	[policy_created_date] [date] NULL,
	[created_date_rc_pol] [date] NULL,
	[damage_premises] [float] NULL,
	[deductible_limit_aggregate] [float] NULL,
	[deductible_limit_amount] [float] NULL,
	[deductible_limit_description] [nvarchar](50) NULL,
	[department_of_transportation_num] [nvarchar](50) NULL,
	[pol_eff_dt] [date] NULL,
	[endorsements_broker_fee] [float] NULL,
	[endorsements_expenses] [float] NULL,
	[endorsements_fees] [float] NULL,
	[endorsements_fronting_charge] [float] NULL,
	[endorsements_gross_total_premium] [float] NULL,
	[endorsements_net_premium] [float] NULL,
	[endorsements_other] [float] NULL,
	[equipment_premium] [float] NULL,
	[expense_mod] [float] NULL,
	[experience_mod] [float] NULL,
	[pol_exp_dt] [date] NULL,
	[extended_reporting_date] [date] NULL,
	[fein] [nvarchar](50) NULL,
	[uw_company] [nvarchar](50) NULL,
	[gic_score] [float] NULL,
	[governing_class_code] [nvarchar](50) NULL,
	[id_for_loads] [nvarchar](100) NULL,
	[insured_address_1] [nvarchar](50) NULL,
	[insured_address_2] [nvarchar](50) NULL,
	[insured_city] [nvarchar](50) NULL,
	[insured_name] [char](150) NULL,
	[insured_state] [nvarchar](50) NULL,
	[insured_zip] [nvarchar](50) NULL,
	[insurer_pct] [float] NULL,
	[insurer_limit_aggregate] [float] NULL,
	[insurer_limit_amount] [float] NULL,
	[insurer_limit_description] [nvarchar](50) NULL,
	[insurer_limit_excess_of] [float] NULL,
	[is_this_tax] [nvarchar](50) NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[policy_last_updated_date] [date] NULL,
	[lead_insurer] [nvarchar](50) NULL,
	[loss_ratio] [float] NULL,
	[mac_id] [nvarchar](50) NULL,
	[marketing_plan] [nvarchar](50) NULL,
	[medical_expense] [nvarchar](50) NULL,
	[model_year] [float] NULL,
	[non_admitted_factor] [nvarchar](50) NULL,
	[occurrence_limit] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[account_id] [nvarchar](50) NULL,
	[original_lob] [nvarchar](50) NULL,
	[other_mod] [float] NULL,
	[package_mod] [float] NULL,
	[personal_advertising_injury] [nvarchar](50) NULL,
	[policy_cancel_date] [nvarchar](50) NULL,
	[policy_number] [nvarchar](100) NULL,
	[policy_postal_code] [nvarchar](50) NULL,
	[policy_renewal] [nvarchar](50) NULL,
	[policy_state] [nvarchar](50) NULL,
	[policy_transaction_type] [nvarchar](50) NULL,
	[policy_trigger] [nvarchar](50) NULL,
	[policy_year] [float] NULL,
	[valen_score] [nvarchar](50) NULL,
	[primary_coverage] [nvarchar](50) NULL,
	[processing_layer] [float] NULL,
	[product_code] [nvarchar](50) NULL,
	[product_comp_works] [nvarchar](50) NULL,
	[program_id] [nvarchar](50) NULL,
	[program_name_link] [nvarchar](50) NULL,
	[retro_date] [date] NULL,
	[sir_limit_aggregate] [float] NULL,
	[sir_limit_amount] [float] NULL,
	[schedule_mod] [float] NULL,
	[program] [nvarchar](50) NULL,
	[sub_producer_address] [nvarchar](100) NULL,
	[sub_producer_commission_char] [nvarchar](100) NULL,
	[sub_producer] [nvarchar](100) NULL,
	[sub_producer_number] [nvarchar](100) NULL,
	[total_broker_fee] [float] NULL,
	[total_expenses] [float] NULL,
	[total_fees] [float] NULL,
	[total_fronting_charge] [float] NULL,
	[total_gross_total_premium] [float] NULL,
	[total_net_premium] [float] NULL,
	[total_other] [float] NULL,
	[type] [nvarchar](50) NULL,
	[xlia_score] [float] NULL,
	[xls_score] [float] NULL,
	[agent_name] [nvarchar](50) NULL,
	[agent_state_code] [nvarchar](50) NULL,
	[agent_zip_code] [nvarchar](50) NULL,
	[aggregate_limit_premops] [float] NULL,
	[aggregate_limit_products] [float] NULL,
	[branch] [nvarchar](50) NULL,
	[business_type_code] [nvarchar](50) NULL,
	[cancel_flag] [nvarchar](50) NULL,
	[consent_to_rate_flag] [nvarchar](50) NULL,
	[employee_count_initial] [float] NULL,
	[location_count_initial] [float] NULL,
	[mod_final] [float] NULL,
	[multiline_policy] [nvarchar](50) NULL,
	[new_renew_flag] [float] NULL,
	[policy_vehicles] [float] NULL,
	[premium_mininum_flag] [nvarchar](50) NULL,
	[quote_policy_term_number] [nvarchar](50) NULL,
	[quote_term_effective_date] [date] NULL,
	[rating_factor_experience_liab] [float] NULL,
	[rating_factor_experience_pd] [float] NULL,
	[rating_tier] [nvarchar](50) NULL,
	[score_id] [nvarchar](50) NULL,
	[sic_code] [nvarchar](50) NULL,
	[term_termination_date] [date] NULL,
	[underwriter] [nvarchar](50) NULL,
	[written_premium_final] [float] NULL,
	[written_premium_initial] [float] NULL,
	[policy_currency_iso_code] [nvarchar](50) NULL,
	[original_policy_inception_date] [date] NULL,
	[sub_producer_commission] [float] NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [policy_import_01](
	[apd_premium] [float] NULL,
	[axa_broker] [nvarchar](50) NULL,
	[axa_other_mod_2] [float] NULL,
	[axa_other_mod_3] [float] NULL,
	[axa_other_mod_4] [float] NULL,
	[aggregate_limit] [float] NULL,
	[annual_broker_fee] [float] NULL,
	[annual_expenses] [float] NULL,
	[annual_fees] [float] NULL,
	[annual_fronting_charge] [float] NULL,
	[annual_gross_total_premium] [float] NULL,
	[annual_net_premium] [float] NULL,
	[annual_other] [float] NULL,
	[assigned_risk] [nvarchar](50) NULL,
	[auto_liability_premium] [float] NULL,
	[billing_code] [nvarchar](50) NULL,
	[broker] [nvarchar](50) NULL,
	[ca_territory_credit] [float] NULL,
	[class_code_description] [char](200) NULL,
	[comments] [char](4000) NULL,
	[commission_contribution] [nvarchar](50) NULL,
	[created_by] [nvarchar](50) NULL,
	[created_by_data_process] [nvarchar](50) NULL,
	[policy_created_date] [date] NULL,
	[created_date_rc_pol] [date] NULL,
	[damage_premises] [float] NULL,
	[deductible_limit_aggregate] [float] NULL,
	[deductible_limit_amount] [float] NULL,
	[deductible_limit_description] [nvarchar](50) NULL,
	[department_of_transportation_num] [nvarchar](50) NULL,
	[pol_eff_dt] [date] NULL,
	[endorsements_broker_fee] [float] NULL,
	[endorsements_expenses] [float] NULL,
	[endorsements_fees] [float] NULL,
	[endorsements_fronting_charge] [float] NULL,
	[endorsements_gross_total_premium] [float] NULL,
	[endorsements_net_premium] [float] NULL,
	[endorsements_other] [float] NULL,
	[equipment_premium] [float] NULL,
	[expense_mod] [float] NULL,
	[experience_mod] [float] NULL,
	[pol_exp_dt] [date] NULL,
	[extended_reporting_date] [date] NULL,
	[fein] [nvarchar](50) NULL,
	[uw_company] [nvarchar](50) NULL,
	[gic_score] [float] NULL,
	[governing_class_code] [nvarchar](50) NULL,
	[id_for_loads] [nvarchar](100) NULL,
	[insured_address_1] [nvarchar](50) NULL,
	[insured_address_2] [nvarchar](50) NULL,
	[insured_city] [nvarchar](50) NULL,
	[insured_name] [char](150) NULL,
	[insured_state] [nvarchar](50) NULL,
	[insured_zip] [nvarchar](50) NULL,
	[insurer_pct] [float] NULL,
	[insurer_limit_aggregate] [float] NULL,
	[insurer_limit_amount] [float] NULL,
	[insurer_limit_description] [nvarchar](50) NULL,
	[insurer_limit_excess_of] [float] NULL,
	[is_this_tax] [nvarchar](50) NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[policy_last_updated_date] [date] NULL,
	[lead_insurer] [nvarchar](50) NULL,
	[loss_ratio] [float] NULL,
	[mac_id] [nvarchar](50) NULL,
	[marketing_plan] [nvarchar](50) NULL,
	[medical_expense] [nvarchar](50) NULL,
	[model_year] [float] NULL,
	[non_admitted_factor] [nvarchar](50) NULL,
	[occurrence_limit] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[account_id] [nvarchar](50) NULL,
	[original_lob] [nvarchar](50) NULL,
	[other_mod] [float] NULL,
	[package_mod] [float] NULL,
	[personal_advertising_injury] [nvarchar](50) NULL,
	[policy_cancel_date] [nvarchar](50) NULL,
	[policy_number] [nvarchar](100) NULL,
	[policy_postal_code] [nvarchar](50) NULL,
	[policy_renewal] [nvarchar](50) NULL,
	[policy_state] [nvarchar](50) NULL,
	[policy_transaction_type] [nvarchar](50) NULL,
	[policy_trigger] [nvarchar](50) NULL,
	[policy_year] [float] NULL,
	[valen_score] [nvarchar](50) NULL,
	[primary_coverage] [nvarchar](50) NULL,
	[processing_layer] [float] NULL,
	[product_code] [nvarchar](50) NULL,
	[product_comp_works] [nvarchar](50) NULL,
	[program_id] [nvarchar](50) NULL,
	[program_name_link] [nvarchar](50) NULL,
	[retro_date] [date] NULL,
	[sir_limit_aggregate] [float] NULL,
	[sir_limit_amount] [float] NULL,
	[schedule_mod] [float] NULL,
	[program] [nvarchar](50) NULL,
	[sub_producer_address] [nvarchar](100) NULL,
	[sub_producer_commission_char] [nvarchar](100) NULL,
	[sub_producer] [nvarchar](100) NULL,
	[sub_producer_number] [nvarchar](100) NULL,
	[total_broker_fee] [float] NULL,
	[total_expenses] [float] NULL,
	[total_fees] [float] NULL,
	[total_fronting_charge] [float] NULL,
	[total_gross_total_premium] [float] NULL,
	[total_net_premium] [float] NULL,
	[total_other] [float] NULL,
	[type] [nvarchar](50) NULL,
	[xlia_score] [float] NULL,
	[xls_score] [float] NULL,
	[agent_name] [nvarchar](50) NULL,
	[agent_state_code] [nvarchar](50) NULL,
	[agent_zip_code] [nvarchar](50) NULL,
	[aggregate_limit_premops] [float] NULL,
	[aggregate_limit_products] [float] NULL,
	[branch] [nvarchar](50) NULL,
	[business_type_code] [nvarchar](50) NULL,
	[cancel_flag] [nvarchar](50) NULL,
	[consent_to_rate_flag] [nvarchar](50) NULL,
	[employee_count_initial] [float] NULL,
	[location_count_initial] [float] NULL,
	[mod_final] [float] NULL,
	[multiline_policy] [nvarchar](50) NULL,
	[new_renew_flag] [float] NULL,
	[policy_vehicles] [float] NULL,
	[premium_mininum_flag] [nvarchar](50) NULL,
	[quote_policy_term_number] [nvarchar](50) NULL,
	[quote_term_effective_date] [date] NULL,
	[rating_factor_experience_liab] [float] NULL,
	[rating_factor_experience_pd] [float] NULL,
	[rating_tier] [nvarchar](50) NULL,
	[score_id] [nvarchar](50) NULL,
	[sic_code] [nvarchar](50) NULL,
	[term_termination_date] [date] NULL,
	[underwriter] [nvarchar](50) NULL,
	[written_premium_final] [float] NULL,
	[written_premium_initial] [float] NULL,
	[policy_currency_iso_code] [nvarchar](50) NULL,
	[original_policy_inception_date] [date] NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [policy_import_02](
	[as_of] [date] NULL,
	[book] [nvarchar](50) NULL,
	[account_name] [nvarchar](50) NULL,
	[specialty_program] [nvarchar](50) NULL,
	[carrier] [nvarchar](50) NULL,
	[LOB] [nvarchar](50) NULL,
	[apd_premium] [float] NULL,
	[axa_broker] [nvarchar](50) NULL,
	[axa_other_mod_2] [float] NULL,
	[axa_other_mod_3] [float] NULL,
	[axa_other_mod_4] [float] NULL,
	[aggregate_limit] [float] NULL,
	[annual_broker_fee] [float] NULL,
	[annual_expenses] [float] NULL,
	[annual_fees] [float] NULL,
	[annual_fronting_charge] [float] NULL,
	[annual_gross_total_premium] [float] NULL,
	[annual_net_premium] [float] NULL,
	[annual_other] [float] NULL,
	[assigned_risk] [nvarchar](50) NULL,
	[auto_liability_premium] [float] NULL,
	[billing_code] [nvarchar](50) NULL,
	[broker] [nvarchar](50) NULL,
	[ca_territory_credit] [float] NULL,
	[class_code_description] [char](200) NULL,
	[comments] [char](4000) NULL,
	[commission_contribution] [nvarchar](50) NULL,
	[created_by] [nvarchar](50) NULL,
	[created_by_data_process] [nvarchar](50) NULL,
	[policy_created_date] [date] NULL,
	[created_date_rc_pol] [date] NULL,
	[damage_premises] [float] NULL,
	[deductible_limit_aggregate] [float] NULL,
	[deductible_limit_amount] [float] NULL,
	[deductible_limit_description] [nvarchar](50) NULL,
	[department_of_transportation_num] [nvarchar](50) NULL,
	[pol_eff_dt] [date] NULL,
	[endorsements_broker_fee] [float] NULL,
	[endorsements_expenses] [float] NULL,
	[endorsements_fees] [float] NULL,
	[endorsements_fronting_charge] [float] NULL,
	[endorsements_gross_total_premium] [float] NULL,
	[endorsements_net_premium] [float] NULL,
	[endorsements_other] [float] NULL,
	[equipment_premium] [float] NULL,
	[expense_mod] [float] NULL,
	[experience_mod] [float] NULL,
	[pol_exp_dt] [date] NULL,
	[extended_reporting_date] [date] NULL,
	[fein] [nvarchar](50) NULL,
	[uw_company] [nvarchar](50) NULL,
	[gic_score] [float] NULL,
	[governing_class_code] [nvarchar](50) NULL,
	[id_for_loads] [nvarchar](100) NULL,
	[insured_address_1] [nvarchar](50) NULL,
	[insured_address_2] [nvarchar](50) NULL,
	[insured_city] [nvarchar](50) NULL,
	[insured_name] [char](150) NULL,
	[insured_state] [nvarchar](50) NULL,
	[insured_zip] [nvarchar](50) NULL,
	[insurer_pct] [float] NULL,
	[insurer_limit_aggregate] [float] NULL,
	[insurer_limit_amount] [float] NULL,
	[insurer_limit_description] [nvarchar](50) NULL,
	[insurer_limit_excess_of] [float] NULL,
	[is_this_tax] [nvarchar](50) NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[policy_last_updated_date] [date] NULL,
	[lead_insurer] [nvarchar](50) NULL,
	[loss_ratio] [float] NULL,
	[mac_id] [nvarchar](50) NULL,
	[marketing_plan] [nvarchar](50) NULL,
	[medical_expense] [nvarchar](50) NULL,
	[model_year] [float] NULL,
	[non_admitted_factor] [nvarchar](50) NULL,
	[occurrence_limit] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[account_id] [nvarchar](50) NULL,
	[original_lob] [nvarchar](50) NULL,
	[other_mod] [float] NULL,
	[package_mod] [float] NULL,
	[personal_advertising_injury] [nvarchar](50) NULL,
	[policy_cancel_date] [nvarchar](50) NULL,
	[policy_number] [nvarchar](100) NULL,
	[policy_postal_code] [nvarchar](50) NULL,
	[policy_renewal] [nvarchar](50) NULL,
	[policy_state] [nvarchar](50) NULL,
	[policy_transaction_type] [nvarchar](50) NULL,
	[policy_trigger] [nvarchar](50) NULL,
	[policy_year] [float] NULL,
	[valen_score] [nvarchar](50) NULL,
	[primary_coverage] [nvarchar](50) NULL,
	[processing_layer] [float] NULL,
	[product_code] [nvarchar](50) NULL,
	[product_comp_works] [nvarchar](50) NULL,
	[program_id] [nvarchar](50) NULL,
	[program_name_link] [nvarchar](50) NULL,
	[retro_date] [date] NULL,
	[sir_limit_aggregate] [float] NULL,
	[sir_limit_amount] [float] NULL,
	[schedule_mod] [float] NULL,
	[program] [nvarchar](50) NULL,
	[sub_producer_address] [nvarchar](100) NULL,
	[sub_producer_commission_char] [nvarchar](100) NULL,
	[sub_producer] [nvarchar](100) NULL,
	[sub_producer_number] [nvarchar](100) NULL,
	[total_broker_fee] [float] NULL,
	[total_expenses] [float] NULL,
	[total_fees] [float] NULL,
	[total_fronting_charge] [float] NULL,
	[total_gross_total_premium] [float] NULL,
	[total_net_premium] [float] NULL,
	[total_other] [float] NULL,
	[type] [nvarchar](50) NULL,
	[xlia_score] [float] NULL,
	[xls_score] [float] NULL,
	[agent_name] [nvarchar](50) NULL,
	[agent_state_code] [nvarchar](50) NULL,
	[agent_zip_code] [nvarchar](50) NULL,
	[aggregate_limit_premops] [float] NULL,
	[aggregate_limit_products] [float] NULL,
	[branch] [nvarchar](50) NULL,
	[business_type_code] [nvarchar](50) NULL,
	[cancel_flag] [nvarchar](50) NULL,
	[consent_to_rate_flag] [nvarchar](50) NULL,
	[employee_count_initial] [float] NULL,
	[location_count_initial] [float] NULL,
	[mod_final] [float] NULL,
	[multiline_policy] [nvarchar](50) NULL,
	[new_renew_flag] [float] NULL,
	[policy_vehicles] [float] NULL,
	[premium_mininum_flag] [nvarchar](50) NULL,
	[quote_policy_term_number] [nvarchar](50) NULL,
	[quote_term_effective_date] [date] NULL,
	[rating_factor_experience_liab] [float] NULL,
	[rating_factor_experience_pd] [float] NULL,
	[rating_tier] [nvarchar](50) NULL,
	[score_id] [nvarchar](50) NULL,
	[sic_code] [nvarchar](50) NULL,
	[term_termination_date] [date] NULL,
	[underwriter] [nvarchar](50) NULL,
	[written_premium_final] [float] NULL,
	[written_premium_initial] [float] NULL,
	[policy_currency_iso_code] [nvarchar](50) NULL,
	[original_policy_inception_date] [date] NULL,
	[sub_producer_commission] [float] NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [premium](
	[as_of] [date] NULL,
	[book] [nvarchar](50) NULL,
	[account_name] [nvarchar](50) NULL,
	[specialty_program] [nvarchar](50) NULL,
	[carrier] [nvarchar](50) NULL,
	[LOB] [nvarchar](50) NULL,
	[al_premium] [float] NULL,
	[al_or_apd_breakout] [nvarchar](50) NULL,
	[apd_premium] [float] NULL,
	[arborist_limit] [float] NULL,
	[assigned_risk] [nvarchar](50) NULL,
	[auto_liability_premium] [float] NULL,
	[base_zone] [nvarchar](50) NULL,
	[blanket_waiver] [nvarchar](50) NULL,
	[broadened_cov_for_contractors] [nvarchar](50) NULL,
	[building_deductible] [float] NULL,
	[building_limit] [float] NULL,
	[business_income_annual_premium] [float] NULL,
	[business_income_deductible] [float] NULL,
	[business_income_limit] [float] NULL,
	[business_income_transaction_prem] [float] NULL,
	[ca_territory_credit] [float] NULL,
	[cat_coverage] [nvarchar](50) NULL,
	[calculation_type_broker_fee] [nvarchar](50) NULL,
	[calculation_type_fees] [nvarchar](50) NULL,
	[calculation_type_fronting_charge] [nvarchar](50) NULL,
	[calculation_type_other] [nvarchar](50) NULL,
	[calculation_type_tax] [nvarchar](50) NULL,
	[cancel_flag] [nvarchar](50) NULL,
	[certified_acts_of_terrorism] [nvarchar](50) NULL,
	[class_csp] [nvarchar](50) NULL,
	[governing_class_code] [nvarchar](50) NULL,
	[consent_to_rate] [nvarchar](50) NULL,
	[construction_class] [nvarchar](50) NULL,
	[construction_class_ii] [nvarchar](50) NULL,
	[contact_email] [nvarchar](50) NULL,
	[contact_name] [nvarchar](50) NULL,
	[contact_phone_number] [nvarchar](50) NULL,
	[contents_deductible] [float] NULL,
	[contents_limit] [float] NULL,
	[coverage] [nvarchar](50) NULL,
	[created_by] [nvarchar](50) NULL,
	[created_date] [date] NULL,
	[created_date_rc_prem] [date] NULL,
	[dot_number] [nvarchar](50) NULL,
	[damage_to_rented_premises] [nvarchar](50) NULL,
	[deductible] [float] NULL,
	[ded_amnt_ded_1] [float] NULL,
	[ded_amnt_ded_14] [float] NULL,
	[ded_amnt_ded_15] [float] NULL,
	[ded_amnt_ded_16] [float] NULL,
	[ded_amnt_ded_2] [float] NULL,
	[ded_amnt_ded_3] [float] NULL,
	[ded_amnt_ded_4] [float] NULL,
	[ded_amnt_ded_5] [float] NULL,
	[ded_amnt_ded_6] [float] NULL,
	[ded_desc_ded_1] [char](100) NULL,
	[ded_desc_ded_14] [char](100) NULL,
	[ded_desc_ded_15] [char](100) NULL,
	[ded_desc_ded_16] [char](100) NULL,
	[ded_desc_ded_2] [char](100) NULL,
	[ded_desc_ded_3] [char](100) NULL,
	[ded_desc_ded_4] [char](100) NULL,
	[ded_desc_ded_5] [char](100) NULL,
	[ded_desc_ded_6] [char](100) NULL,
	[earthquake_ded] [float] NULL,
	[expense_mod] [float] NULL,
	[experience_mod] [float] NULL,
	[fein] [nvarchar](50) NULL,
	[firm] [nvarchar](50) NULL,
	[class_code] [nvarchar](50) NULL,
	[guaranteed_cost_flag] [nvarchar](50) NULL,
	[id_used_for_data_loads] [nvarchar](100) NULL,
	[insured_address_1] [char](100) NULL,
	[insured_address_2] [char](100) NULL,
	[insured_city] [nvarchar](50) NULL,
	[insured_name] [nvarchar](150) NULL,
	[insured_state] [nvarchar](50) NULL,
	[insured_zip] [nvarchar](50) NULL,
	[insurer] [nvarchar](50) NULL,
	[is_renewal] [nvarchar](50) NULL,
	[is_this_tax] [nvarchar](50) NULL,
	[lcm] [float] NULL,
	[landscape_or_arborist_services] [nvarchar](50) NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[last_updated_date] [date] NULL,
	[limit_amnt_limit_1] [float] NULL,
	[limit_amnt_limit_10] [float] NULL,
	[limit_amnt_limit_11] [float] NULL,
	[limit_amnt_limit_12] [float] NULL,
	[limit_amnt_limit_13] [float] NULL,
	[limit_amnt_limit_2] [float] NULL,
	[limit_amnt_limit_3] [float] NULL,
	[limit_amnt_limit_4] [float] NULL,
	[limit_amnt_limit_5] [float] NULL,
	[limit_amnt_limit_6] [float] NULL,
	[limit_amnt_limit_7] [float] NULL,
	[limit_amnt_limit_8] [float] NULL,
	[limit_amnt_limit_9] [float] NULL,
	[limit_desc_limit_1] [char](100) NULL,
	[limit_desc_limit_10] [char](100) NULL,
	[limit_desc_limit_11] [char](100) NULL,
	[limit_desc_limit_12] [char](100) NULL,
	[limit_desc_limit_13] [char](100) NULL,
	[limit_desc_limit_2] [char](100) NULL,
	[limit_desc_limit_3] [char](100) NULL,
	[limit_desc_limit_4] [char](100) NULL,
	[limit_desc_limit_5] [char](100) NULL,
	[limit_desc_limit_6] [char](100) NULL,
	[limit_desc_limit_7] [char](100) NULL,
	[limit_desc_limit_8] [char](100) NULL,
	[limit_desc_limit_9] [char](100) NULL,
	[load_notes] [char](200) NULL,
	[loss_cost_original] [nvarchar](50) NULL,
	[mod_sate] [nvarchar](50) NULL,
	[model_description] [char](100) NULL,
	[model_make] [nvarchar](50) NULL,
	[model_year_char] [nvarchar](50) NULL,
	[nbfu_fire_protection_class] [nvarchar](50) NULL,
	[occupancy_type] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[other_mod] [float] NULL,
	[package_mod] [float] NULL,
	[percentage_broker_fee] [float] NULL,
	[percentage_fees] [float] NULL,
	[percentage_fronting_charge] [float] NULL,
	[percentage_other] [float] NULL,
	[percentage_tax] [float] NULL,
	[policy_cancel_date] [date] NULL,
	[policy_id_for_load] [nvarchar](100) NULL,
	[policy_number] [nvarchar](50) NULL,
	[pol_eff_dt] [date] NULL,
	[premium_transaction_type] [nvarchar](100) NULL,
	[premium_vehicle_type] [nvarchar](50) NULL,
	[primary_coverage] [nvarchar](50) NULL,
	[radius_class] [nvarchar](50) NULL,
	[raiting_tier] [nvarchar](50) NULL,
	[rate] [float] NULL,
	[ssu] [nvarchar](50) NULL,
	[schedule_mod] [float] NULL,
	[source_trans_type] [nvarchar](50) NULL,
	[special_waiver] [nvarchar](50) NULL,
	[stated_amount] [float] NULL,
	[stop_gap_EL_cov] [nvarchar](50) NULL,
	[stop_gap_limits] [float] NULL,
	[subline_code] [nvarchar](50) NULL,
	[subline_description] [char](100) NULL,
	[terminal_zone] [nvarchar](50) NULL,
	[territory] [nvarchar](50) NULL,
	[time_element_limit] [float] NULL,
	[total_broker_fee] [float] NULL,
	[total_fees] [float] NULL,
	[total_fronting_charge] [float] NULL,
	[total_net_premium] [float] NULL,
	[total_other] [float] NULL,
	[total_paid] [float] NULL,
	[total_tax] [float] NULL,
	[unit_business_use] [nvarchar](50) NULL,
	[unit_garage_zip] [nvarchar](50) NULL,
	[unit_number_char] [nvarchar](50) NULL,
	[vehicle_class] [nvarchar](50) NULL,
	[vin] [nvarchar](50) NULL,
	[wind_hail_deductible] [float] NULL,
	[year_built] [float] NULL,
	[coverage_code] [nvarchar](50) NULL,
	[coverage_effective_date] [date] NULL,
	[coverage_expiration_date] [date] NULL,
	[coverage_symbol] [nvarchar](50) NULL,
	[coverage_type_code] [nvarchar](50) NULL,
	[deductible_amount] [float] NULL,
	[limit_amount_final] [float] NULL,
	[limit_amount_initial] [float] NULL,
	[original_coverage_code] [nvarchar](50) NULL,
	[original_coverage_desc] [char](100) NULL,
	[premium_manual_final] [float] NULL,
	[premium_manual_initial] [float] NULL,
	[premium_written_final] [float] NULL,
	[premium_written_initial] [float] NULL,
	[rating_company_factor] [float] NULL,
	[rating_tier_factor] [float] NULL,
	[schedule_mod_factor] [float] NULL,
	[territory_factor] [float] NULL,
	[transaction_type] [nvarchar](50) NULL,
	[unit_type_code] [nvarchar](50) NULL,
	[premium_created_date_char] [nvarchar](50) NULL,
	[linked_policy_id] [nvarchar](200) NULL,
	[total_gross_total_premium] [float] NULL,
	[total_outstanding] [float] NULL,
	[model_year] [float] NULL,
	[unit_number] [float] NULL,
	[premium_created_date] [date] NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [premium_import_01](
	[al_premium] [float] NULL,
	[al_or_apd_breakout] [nvarchar](50) NULL,
	[apd_premium] [float] NULL,
	[arborist_limit] [float] NULL,
	[assigned_risk] [nvarchar](50) NULL,
	[auto_liability_premium] [float] NULL,
	[base_zone] [nvarchar](50) NULL,
	[blanket_waiver] [nvarchar](50) NULL,
	[broadened_cov_for_contractors] [nvarchar](50) NULL,
	[building_deductible] [float] NULL,
	[building_limit] [float] NULL,
	[business_income_annual_premium] [float] NULL,
	[business_income_deductible] [float] NULL,
	[business_income_limit] [float] NULL,
	[business_income_transaction_prem] [float] NULL,
	[ca_territory_credit] [float] NULL,
	[cat_coverage] [nvarchar](50) NULL,
	[calculation_type_broker_fee] [nvarchar](50) NULL,
	[calculation_type_fees] [nvarchar](50) NULL,
	[calculation_type_fronting_charge] [nvarchar](50) NULL,
	[calculation_type_other] [nvarchar](50) NULL,
	[calculation_type_tax] [nvarchar](50) NULL,
	[cancel_flag] [nvarchar](50) NULL,
	[certified_acts_of_terrorism] [nvarchar](50) NULL,
	[class_csp] [nvarchar](50) NULL,
	[;verning_class_code] [nvarchar](50) NULL,
	[consent_to_rate] [nvarchar](50) NULL,
	[construction_class] [nvarchar](50) NULL,
	[construction_class_ii] [nvarchar](50) NULL,
	[contact_email] [nvarchar](50) NULL,
	[contact_name] [nvarchar](50) NULL,
	[contact_phone_number] [nvarchar](50) NULL,
	[contents_deductible] [float] NULL,
	[contents_limit] [float] NULL,
	[coverage] [nvarchar](50) NULL,
	[created_by] [nvarchar](50) NULL,
	[created_date] [date] NULL,
	[created_date_rc_prem] [date] NULL,
	[dot_number] [nvarchar](50) NULL,
	[damage_to_rented_premises] [nvarchar](50) NULL,
	[deductible] [float] NULL,
	[ded_amnt_ded_1] [float] NULL,
	[ded_amnt_ded_14] [float] NULL,
	[ded_amnt_ded_15] [float] NULL,
	[ded_amnt_ded_16] [float] NULL,
	[ded_amnt_ded_2] [float] NULL,
	[ded_amnt_ded_3] [float] NULL,
	[ded_amnt_ded_4] [float] NULL,
	[ded_amnt_ded_5] [float] NULL,
	[ded_amnt_ded_6] [float] NULL,
	[ded_desc_ded_1] [char](100) NULL,
	[ded_desc_ded_14] [char](100) NULL,
	[ded_desc_ded_15] [char](100) NULL,
	[ded_desc_ded_16] [char](100) NULL,
	[ded_desc_ded_2] [char](100) NULL,
	[ded_desc_ded_3] [char](100) NULL,
	[ded_desc_ded_4] [char](100) NULL,
	[ded_desc_ded_5] [char](100) NULL,
	[ded_desc_ded_6] [char](100) NULL,
	[earthquake_ded] [float] NULL,
	[expense_mod] [float] NULL,
	[experience_mod] [float] NULL,
	[fein] [nvarchar](50) NULL,
	[firm] [nvarchar](50) NULL,
	[class_code] [nvarchar](50) NULL,
	[guaranteed_cost_flag] [nvarchar](50) NULL,
	[id_used_for_data_loads] [nvarchar](100) NULL,
	[insured_address_1] [char](100) NULL,
	[insured_address_2] [char](100) NULL,
	[insured_city] [nvarchar](50) NULL,
	[insured_name] [nvarchar](150) NULL,
	[insured_state] [nvarchar](50) NULL,
	[insured_zip] [nvarchar](50) NULL,
	[insurer] [nvarchar](50) NULL,
	[is_renewal] [nvarchar](50) NULL,
	[is_this_tax] [nvarchar](50) NULL,
	[lcm] [float] NULL,
	[landscape_or_arborist_services] [nvarchar](50) NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[last_updated_date] [date] NULL,
	[limit_amnt_limit_1] [float] NULL,
	[limit_amnt_limit_10] [float] NULL,
	[limit_amnt_limit_11] [float] NULL,
	[limit_amnt_limit_12] [float] NULL,
	[limit_amnt_limit_13] [float] NULL,
	[limit_amnt_limit_2] [float] NULL,
	[limit_amnt_limit_3] [float] NULL,
	[limit_amnt_limit_4] [float] NULL,
	[limit_amnt_limit_5] [float] NULL,
	[limit_amnt_limit_6] [float] NULL,
	[limit_amnt_limit_7] [float] NULL,
	[limit_amnt_limit_8] [float] NULL,
	[limit_amnt_limit_9] [float] NULL,
	[limit_desc_limit_1] [char](100) NULL,
	[limit_desc_limit_10] [char](100) NULL,
	[limit_desc_limit_11] [char](100) NULL,
	[limit_desc_limit_12] [char](100) NULL,
	[limit_desc_limit_13] [char](100) NULL,
	[limit_desc_limit_2] [char](100) NULL,
	[limit_desc_limit_3] [char](100) NULL,
	[limit_desc_limit_4] [char](100) NULL,
	[limit_desc_limit_5] [char](100) NULL,
	[limit_desc_limit_6] [char](100) NULL,
	[limit_desc_limit_7] [char](100) NULL,
	[limit_desc_limit_8] [char](100) NULL,
	[limit_desc_limit_9] [char](100) NULL,
	[load_notes] [char](200) NULL,
	[loss_cost_original] [nvarchar](50) NULL,
	[mod_sate] [nvarchar](50) NULL,
	[model_description] [char](100) NULL,
	[model_make] [nvarchar](50) NULL,
	[model_year_char] [nvarchar](50) NULL,
	[nbfu_fire_protection_class] [nvarchar](50) NULL,
	[occupancy_type] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[other_mod] [float] NULL,
	[package_mod] [float] NULL,
	[percentage_broker_fee] [float] NULL,
	[percentage_fees] [float] NULL,
	[percentage_fronting_charge] [float] NULL,
	[percentage_other] [float] NULL,
	[percentage_tax] [float] NULL,
	[policy_cancel_date] [date] NULL,
	[policy_id_for_load] [nvarchar](100) NULL,
	[policy_number] [nvarchar](50) NULL,
	[pol_eff_dt] [date] NULL,
	[premium_transaction_type] [nvarchar](100) NULL,
	[premium_vehicle_type] [nvarchar](50) NULL,
	[primary_coverage] [nvarchar](50) NULL,
	[radius_class] [nvarchar](50) NULL,
	[raiting_tier] [nvarchar](50) NULL,
	[rate] [float] NULL,
	[ssu] [nvarchar](50) NULL,
	[schedule_mod] [float] NULL,
	[source_trans_type] [nvarchar](50) NULL,
	[special_waiver] [nvarchar](50) NULL,
	[stated_amount] [float] NULL,
	[stop_gap_EL_cov] [nvarchar](50) NULL,
	[stop_gap_limits] [float] NULL,
	[subline_code] [nvarchar](50) NULL,
	[subline_description] [char](100) NULL,
	[terminal_zone] [nvarchar](50) NULL,
	[territory] [nvarchar](50) NULL,
	[time_element_limit] [float] NULL,
	[total_broker_fee] [float] NULL,
	[total_fees] [float] NULL,
	[total_fronting_charge] [float] NULL,
	[total_net_premium] [float] NULL,
	[total_other] [float] NULL,
	[total_paid] [float] NULL,
	[total_tax] [float] NULL,
	[unit_business_use] [nvarchar](50) NULL,
	[unit_garage_zip] [nvarchar](50) NULL,
	[unit_number_char] [nvarchar](50) NULL,
	[vehicle_class] [nvarchar](50) NULL,
	[vin] [nvarchar](50) NULL,
	[wind_hail_deductible] [float] NULL,
	[year_built] [float] NULL,
	[coverage_code] [nvarchar](50) NULL,
	[coverage_effective_date] [date] NULL,
	[coverage_expiration_date] [date] NULL,
	[coverage_symbol] [nvarchar](50) NULL,
	[coverage_type_code] [nvarchar](50) NULL,
	[deductible_amount] [float] NULL,
	[limit_amount_final] [float] NULL,
	[limit_amount_initial] [float] NULL,
	[original_coverage_code] [nvarchar](50) NULL,
	[original_coverage_desc] [char](100) NULL,
	[premium_manual_final] [float] NULL,
	[premium_manual_initial] [float] NULL,
	[premium_written_final] [float] NULL,
	[premium_written_initial] [float] NULL,
	[rating_company_factor] [float] NULL,
	[rating_tier_factor] [float] NULL,
	[schedule_mod_factor] [float] NULL,
	[territory_factor] [float] NULL,
	[transaction_type] [nvarchar](50) NULL,
	[unit_type_code] [nvarchar](50) NULL,
	[premium_created_date_char] [nvarchar](50) NULL,
	[linked_policy_id] [nvarchar](50) NULL,
	[total_gross_total_premium] [float] NULL,
	[total_outstanding] [float] NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [premium_import_02](
	[as_of] [date] NULL,
	[book] [nvarchar](50) NULL,
	[account_name] [nvarchar](50) NULL,
	[specialty_program] [nvarchar](50) NULL,
	[carrier] [nvarchar](50) NULL,
	[LOB] [nvarchar](50) NULL,
	[al_premium] [float] NULL,
	[al_or_apd_breakout] [nvarchar](50) NULL,
	[apd_premium] [float] NULL,
	[arborist_limit] [float] NULL,
	[assigned_risk] [nvarchar](50) NULL,
	[auto_liability_premium] [float] NULL,
	[base_zone] [nvarchar](50) NULL,
	[blanket_waiver] [nvarchar](50) NULL,
	[broadened_cov_for_contractors] [nvarchar](50) NULL,
	[building_deductible] [float] NULL,
	[building_limit] [float] NULL,
	[business_income_annual_premium] [float] NULL,
	[business_income_deductible] [float] NULL,
	[business_income_limit] [float] NULL,
	[business_income_transaction_prem] [float] NULL,
	[ca_territory_credit] [float] NULL,
	[cat_coverage] [nvarchar](50) NULL,
	[calculation_type_broker_fee] [nvarchar](50) NULL,
	[calculation_type_fees] [nvarchar](50) NULL,
	[calculation_type_fronting_charge] [nvarchar](50) NULL,
	[calculation_type_other] [nvarchar](50) NULL,
	[calculation_type_tax] [nvarchar](50) NULL,
	[cancel_flag] [nvarchar](50) NULL,
	[certified_acts_of_terrorism] [nvarchar](50) NULL,
	[class_csp] [nvarchar](50) NULL,
	[;verning_class_code] [nvarchar](50) NULL,
	[consent_to_rate] [nvarchar](50) NULL,
	[construction_class] [nvarchar](50) NULL,
	[construction_class_ii] [nvarchar](50) NULL,
	[contact_email] [nvarchar](50) NULL,
	[contact_name] [nvarchar](50) NULL,
	[contact_phone_number] [nvarchar](50) NULL,
	[contents_deductible] [float] NULL,
	[contents_limit] [float] NULL,
	[coverage] [nvarchar](50) NULL,
	[created_by] [nvarchar](50) NULL,
	[created_date] [date] NULL,
	[created_date_rc_prem] [date] NULL,
	[dot_number] [nvarchar](50) NULL,
	[damage_to_rented_premises] [nvarchar](50) NULL,
	[deductible] [float] NULL,
	[ded_amnt_ded_1] [float] NULL,
	[ded_amnt_ded_14] [float] NULL,
	[ded_amnt_ded_15] [float] NULL,
	[ded_amnt_ded_16] [float] NULL,
	[ded_amnt_ded_2] [float] NULL,
	[ded_amnt_ded_3] [float] NULL,
	[ded_amnt_ded_4] [float] NULL,
	[ded_amnt_ded_5] [float] NULL,
	[ded_amnt_ded_6] [float] NULL,
	[ded_desc_ded_1] [char](100) NULL,
	[ded_desc_ded_14] [char](100) NULL,
	[ded_desc_ded_15] [char](100) NULL,
	[ded_desc_ded_16] [char](100) NULL,
	[ded_desc_ded_2] [char](100) NULL,
	[ded_desc_ded_3] [char](100) NULL,
	[ded_desc_ded_4] [char](100) NULL,
	[ded_desc_ded_5] [char](100) NULL,
	[ded_desc_ded_6] [char](100) NULL,
	[earthquake_ded] [float] NULL,
	[expense_mod] [float] NULL,
	[experience_mod] [float] NULL,
	[fein] [nvarchar](50) NULL,
	[firm] [nvarchar](50) NULL,
	[class_code] [nvarchar](50) NULL,
	[guaranteed_cost_flag] [nvarchar](50) NULL,
	[id_used_for_data_loads] [nvarchar](100) NULL,
	[insured_address_1] [char](100) NULL,
	[insured_address_2] [char](100) NULL,
	[insured_city] [nvarchar](50) NULL,
	[insured_name] [nvarchar](150) NULL,
	[insured_state] [nvarchar](50) NULL,
	[insured_zip] [nvarchar](50) NULL,
	[insurer] [nvarchar](50) NULL,
	[is_renewal] [nvarchar](50) NULL,
	[is_this_tax] [nvarchar](50) NULL,
	[lcm] [float] NULL,
	[landscape_or_arborist_services] [nvarchar](50) NULL,
	[last_updated_by] [nvarchar](50) NULL,
	[last_updated_date] [date] NULL,
	[limit_amnt_limit_1] [float] NULL,
	[limit_amnt_limit_10] [float] NULL,
	[limit_amnt_limit_11] [float] NULL,
	[limit_amnt_limit_12] [float] NULL,
	[limit_amnt_limit_13] [float] NULL,
	[limit_amnt_limit_2] [float] NULL,
	[limit_amnt_limit_3] [float] NULL,
	[limit_amnt_limit_4] [float] NULL,
	[limit_amnt_limit_5] [float] NULL,
	[limit_amnt_limit_6] [float] NULL,
	[limit_amnt_limit_7] [float] NULL,
	[limit_amnt_limit_8] [float] NULL,
	[limit_amnt_limit_9] [float] NULL,
	[limit_desc_limit_1] [char](100) NULL,
	[limit_desc_limit_10] [char](100) NULL,
	[limit_desc_limit_11] [char](100) NULL,
	[limit_desc_limit_12] [char](100) NULL,
	[limit_desc_limit_13] [char](100) NULL,
	[limit_desc_limit_2] [char](100) NULL,
	[limit_desc_limit_3] [char](100) NULL,
	[limit_desc_limit_4] [char](100) NULL,
	[limit_desc_limit_5] [char](100) NULL,
	[limit_desc_limit_6] [char](100) NULL,
	[limit_desc_limit_7] [char](100) NULL,
	[limit_desc_limit_8] [char](100) NULL,
	[limit_desc_limit_9] [char](100) NULL,
	[load_notes] [char](200) NULL,
	[loss_cost_original] [nvarchar](50) NULL,
	[mod_sate] [nvarchar](50) NULL,
	[model_description] [char](100) NULL,
	[model_make] [nvarchar](50) NULL,
	[model_year_char] [nvarchar](50) NULL,
	[nbfu_fire_protection_class] [nvarchar](50) NULL,
	[occupancy_type] [nvarchar](50) NULL,
	[organization] [nvarchar](50) NULL,
	[other_mod] [float] NULL,
	[package_mod] [float] NULL,
	[percentage_broker_fee] [float] NULL,
	[percentage_fees] [float] NULL,
	[percentage_fronting_charge] [float] NULL,
	[percentage_other] [float] NULL,
	[percentage_tax] [float] NULL,
	[policy_cancel_date] [date] NULL,
	[policy_id_for_load] [nvarchar](100) NULL,
	[policy_number] [nvarchar](50) NULL,
	[pol_eff_dt] [date] NULL,
	[premium_transaction_type] [nvarchar](100) NULL,
	[premium_vehicle_type] [nvarchar](50) NULL,
	[primary_coverage] [nvarchar](50) NULL,
	[radius_class] [nvarchar](50) NULL,
	[raiting_tier] [nvarchar](50) NULL,
	[rate] [float] NULL,
	[ssu] [nvarchar](50) NULL,
	[schedule_mod] [float] NULL,
	[source_trans_type] [nvarchar](50) NULL,
	[special_waiver] [nvarchar](50) NULL,
	[stated_amount] [float] NULL,
	[stop_gap_EL_cov] [nvarchar](50) NULL,
	[stop_gap_limits] [float] NULL,
	[subline_code] [nvarchar](50) NULL,
	[subline_description] [char](100) NULL,
	[terminal_zone] [nvarchar](50) NULL,
	[territory] [nvarchar](50) NULL,
	[time_element_limit] [float] NULL,
	[total_broker_fee] [float] NULL,
	[total_fees] [float] NULL,
	[total_fronting_charge] [float] NULL,
	[total_net_premium] [float] NULL,
	[total_other] [float] NULL,
	[total_paid] [float] NULL,
	[total_tax] [float] NULL,
	[unit_business_use] [nvarchar](50) NULL,
	[unit_garage_zip] [nvarchar](50) NULL,
	[unit_number_char] [nvarchar](50) NULL,
	[vehicle_class] [nvarchar](50) NULL,
	[vin] [nvarchar](50) NULL,
	[wind_hail_deductible] [float] NULL,
	[year_built] [float] NULL,
	[coverage_code] [nvarchar](50) NULL,
	[coverage_effective_date] [date] NULL,
	[coverage_expiration_date] [date] NULL,
	[coverage_symbol] [nvarchar](50) NULL,
	[coverage_type_code] [nvarchar](50) NULL,
	[deductible_amount] [float] NULL,
	[limit_amount_final] [float] NULL,
	[limit_amount_initial] [float] NULL,
	[original_coverage_code] [nvarchar](50) NULL,
	[original_coverage_desc] [char](100) NULL,
	[premium_manual_final] [float] NULL,
	[premium_manual_initial] [float] NULL,
	[premium_written_final] [float] NULL,
	[premium_written_initial] [float] NULL,
	[rating_company_factor] [float] NULL,
	[rating_tier_factor] [float] NULL,
	[schedule_mod_factor] [float] NULL,
	[territory_factor] [float] NULL,
	[transaction_type] [nvarchar](50) NULL,
	[unit_type_code] [nvarchar](50) NULL,
	[premium_created_date_char] [nvarchar](50) NULL,
	[linked_policy_id] [nvarchar](50) NULL,
	[total_gross_total_premium] [float] NULL,
	[total_outstanding] [float] NULL,
	[model_year] [float] NULL,
	[unit_number] [float] NULL,
	[premium_created_date] [date] NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [ref_claims_patch](
	[LOB] [varchar](10) NULL,
	[Claim_Number] [bigint] NULL,
	[Claimant_Sequence] [int] NULL,
	[Party_Update] [varchar](20) NULL,
	[Coverage_Code_Detail] [varchar](255) NULL,
	[Claim_Type] [varchar](10) NULL,
	[ref_claim_number] [nvarchar](50) NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [ref_class_code](
	[LOB] [varchar](50) NULL,
	[class_code] [varchar](20) NULL,
	[class_code_desc] [varchar](300) NULL,
	[auto_size_class] [varchar](60) NULL,
	[auto_size_class_type] [varchar](30) NULL,
	[auto_size_class_weight] [varchar](30) NULL,
	[auto_business_use_class] [varchar](15) NULL,
	[auto_radius_class] [varchar](30) NULL,
	[auto_fleet] [varchar](2) NULL,
	[auto_classification] [varchar](40) NULL,
	[auto_subclassification] [varchar](70) NULL,
	[auto_type] [varchar](30) NULL,
	[auto_subtype] [varchar](30) NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [ref_cov_desc](
	[book] [varchar](25) NULL,
	[LOB] [varchar](20) NULL,
	[TPA] [varchar](50) NULL,
	[descriptors_source_coverage] [varchar](50) NULL,
	[cov_desc] [varchar](50) NULL,
	[cov_desc_detail] [varchar](50) NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [ref_lob](
	[coverage] [varchar](50) NULL,
	[LOB] [varchar](50) NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [ref_marketing_plan](
	[marketing_plan_code] [varchar](10) NULL,
	[marketing_plan] [varchar](100) NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [ref_organization](
	[organization] [varchar](50) NULL,
	[book] [varchar](25) NULL,
	[account_name] [varchar](50) NULL,
	[specialty_program] [varchar](30) NULL,
	[carrier] [varchar](30) NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [sp_premium_triangles](
	[as_of] [date] NULL,
	[book] [varchar](100) NULL,
	[specialty_program] [varchar](100) NULL,
	[carrier] [varchar](100) NULL,
	[pol_eff_yr] [numeric](11, 0) NULL,
	[CAY] [smallint] NULL,
	[CAQ] [varchar](6) NULL,
	[CAY_age] [float] NULL,
	[LOB] [varchar](100) NULL,
	[organization] [varchar](100) NULL,
	[account_id] [varchar](100) NULL,
	[mac_id] [varchar](100) NULL,
	[insured_name] [varchar](255) NULL,
	[policy_number] [varchar](200) NULL,
	[uw_company] [varchar](100) NULL,
	[broker] [varchar](100) NULL,
	[axa_broker] [varchar](100) NULL,
	[billing_code] [varchar](100) NULL,
	[marketing_plan] [varchar](100) NULL,
	[program] [varchar](100) NULL,
	[pol_eff_dt] [date] NULL,
	[pol_exp_dt] [date] NULL,
	[multiline_policy] [varchar](100) NULL,
	[new_renew_flag] [float] NULL,
	[primary_coverage] [varchar](100) NULL,
	[governing_class_code] [varchar](100) NULL,
	[policy_state] [varchar](100) NULL,
	[policy_state_region] [varchar](100) NULL,
	[policy_state_ca_ind] [varchar](100) NULL,
	[policy_zip] [varchar](100) NULL,
	[insured_city] [varchar](100) NULL,
	[insured_state] [varchar](100) NULL,
	[insured_zip] [varchar](100) NULL,
	[valen_score] [varchar](100) NULL,
	[policy_count] [float] NULL,
	[written_premium] [float] NULL,
	[earned_premium] [float] NULL
) ON [PRIMARY]
GO
SET ANSI_NULLS ON
GO
SET QUOTED_IDENTIFIER ON
GO
CREATE TABLE [xref_marketing_plan_to_organization](
	[marketing_plan_code] [varchar](10) NULL,
	[marketing_plan] [varchar](100) NULL,
	[organization] [varchar](50) NULL
) ON [PRIMARY]
GO
